In [1]:
#@title #1. Library { display-mode: "form" }
import os
import re
import math
import warnings
import numpy as np
import pandas as pd
import gspread as gs
import seaborn as sns
from google.colab import files, drive, auth
from google.auth import default
from collections import Counter
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# import gc
# gc.collect(generation=0)

In [2]:
#@title Autenticador en drive { display-mode: "form" }
#@markdown ---
# authorization in drive
drive.mount('/content/drive')
dir_path = '/content/drive/MyDrive/ETB/Codes/2022/Functions/Normalization/'
os.chdir(dir_path)
from func_callout import *
dir_path = '/content/drive/MyDrive/ETB/Data/Actualización bases mensuales ETB Total/'
os.chdir(dir_path)

Mounted at /content/drive


# Mayo

In [ ]:
#@title ###Seleccionar mes para analizar {run: "auto", vertical-output: true }
Mes = 'Mayo' #@param ['Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')


In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['Base Total de Clientes de la Operacion Fija_20220527.xlsx',
 'Retorno_Mayo.xlsx',
 'seguimiento_corte_Mayo.csv',
 'Retorno_Mayo_v2.xlsx']

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Base Total' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633323 entries, 0 to 633322
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   TIPO_DOCUMENTO         633323 non-null  object 
 1   NUMERO_DOCUMENTO       633323 non-null  object 
 2   NOMBRE_CLIENTE         632588 non-null  object 
 3   TELEFONO               620928 non-null  object 
 4   TELEFONO_CONTACTO_1    483227 non-null  float64
 5   TELEFONO_CONTACTO_2    525601 non-null  float64
 6   TELEFONO_CONTACTO_3    320759 non-null  float64
 7   LOCALIDAD              564322 non-null  object 
 8   BARRIO                 547616 non-null  object 
 9   DIRECCION_INSTALACION  633192 non-null  object 
 10  LATITUD                564422 non-null  float64
 11  LONGITUD               564422 non-null  float64
 12  PAQUETE                633323 non-null  object 
 13  PAQUETE_PLAN           598791 non-null  object 
 14  CICLO                  631355 non-nu

In [ ]:
#@markdown ###Carga archivo ETB Total Ultima versión actualizada { run: "auto", vertical-output: true }
Meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
path_etb_total = os.path.join('Data/ETB_Total/'+Year+'/'+Meses[Meses.index(Mes)-1]+'/','')
Nombre_archivo_act = 'Base Total Actualizada'
nombre_completo_act= [i for i in os.listdir(path_etb_total) if Nombre_archivo_act in i][0]
df_t = pd.read_excel(path_etb_total+nombre_completo_act,sheet_name='Consulta1')
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633766 entries, 0 to 633765
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   TIPO_DOCUMENTO         633766 non-null  object 
 1   NUMERO_DOCUMENTO       633766 non-null  object 
 2   NOMBRE_CLIENTE         632783 non-null  object 
 3   TELEFONO               624062 non-null  object 
 4   TELEFONO_CONTACTO_1    481427 non-null  float64
 5   TELEFONO_CONTACTO_2    521560 non-null  float64
 6   TELEFONO_CONTACTO_3    315034 non-null  float64
 7   LOCALIDAD              561002 non-null  object 
 8   BARRIO                 543542 non-null  object 
 9   DIRECCION_INSTALACION  633654 non-null  object 
 10  LATITUD                563027 non-null  float64
 11  LONGITUD               563027 non-null  float64
 12  PAQUETE                633766 non-null  object 
 13  PAQUETE_PLAN           597708 non-null  object 
 14  CICLO                  630756 non-nu

In [ ]:
df_t.shape

(633766, 34)

In [ ]:
df_t[df_t.TELEFONO=='17386429']

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,ESTRATO_USO,celular_1,celular_2,celular_3,C_SEG_UEN,Localidad_zona,tipo_estrato,Tipo_velocidad,grupos,pqr
586089,CC,3087400,ALVARO JOSE BLANDON,17386429,6.017386e+09,3.115692e+09,NaN,KENNEDY,CIUDAD TECHO II,KR 82A 6 37 IN 7 AP 1028,...,4,3.115692e+09,3.115692e+09,3.144409e+09,HOGARES,zona centro sur,alto,Mayor 100,FTTH - Dúo - zona centro sur - alto - Mayor 100,3


In [ ]:
df_t.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_t.reset_index(drop=True, inplace=True)
df_t.shape

(633761, 34)

In [ ]:
df_t_ = df_t.copy()

In [ ]:
df_t_['TELEFONO2'] = 0
df_t_['TELEFONO2'] = df_t_.TELEFONO

In [ ]:
df_m.shape

(633323, 34)

In [ ]:
df_m.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m.reset_index(drop=True, inplace=True)
df_m.shape

(633138, 34)

In [ ]:
dft_st = df_t[df_t_.TELEFONO.isna()]
dft_ct = df_t[df_t_.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc),len(ind_con),len(indx))
# nuevos
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

Registros nuevos: (18706, 34)


In [ ]:
df_New['Emergia_Mayo_nuevo'] = 1
df_t_['Emergia_Mayo_nuevo'] = 0
dfm_st['Emergia_Mayo_nuevo'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#@title cantidad de ragistros con los nuevos mes mayo sin NaNs
dft_ct.shape[0]+df_New.shape[0]

642764

In [ ]:
temporal = pd.DataFrame()
temporal = pd.concat([df_t_, df_New, dfm_st], axis=0)
temporal.shape

(664861, 36)

In [ ]:
dft_ct = temporal[temporal.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# datos = dft_ct[['TELEFONO','NUMERO_DOCUMENTO']].iloc[indx]
# y1 = datos['NUMERO_DOCUMENTO'].tolist()
# y2 = datos['TELEFONO'].tolist()
# indx = df_t.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
df_Eli = dft_ct.drop(indx,axis=0)
print('Registros eliminados: {}'.format(df_Eli.shape))

Registros eliminados: (22037, 36)


In [ ]:
df_Eli = df_Eli[['NUMERO_DOCUMENTO', 'TELEFONO']]
df_Eli['Emergia_Mayo_Bajas'] = 1

In [ ]:
temporal = temporal.merge(df_Eli, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')

In [ ]:
temporal[temporal.Emergia_Mayo_nuevo!=1][temporal.TELEFONO.notna()][temporal.Emergia_Mayo_Bajas!=1].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(602021, 37)

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Retorno' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m_retorno = pd.read_excel(path+nombre_completo)
df_m_retorno.info()

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161075 entries, 0 to 161074
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   NUMERO_DOCUMENTO_CLIENTE  161075 non-null  object 
 1   NUMERO_CONEXION           160561 non-null  float64
 2   Tipificacion              161072 non-null  object 
 3   Venta                     161075 non-null  float64
 4   Gestionable               161075 non-null  float64
dtypes: float64(3), object(2)
memory usage: 6.1+ MB


In [ ]:
df_m_retorno.rename(columns={'NUMERO_CONEXION':'TELEFONO','NUMERO_DOCUMENTO_CLIENTE':'NUMERO_DOCUMENTO'}, inplace=True)
df_m_retorno.shape

(161075, 5)

In [ ]:
df_m_retorno.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m_retorno.shape

(161075, 5)

In [ ]:
df_m_retorno['TELEFONO'] = df_m_retorno['TELEFONO'].astype(str)
df_m_retorno["TELEFONO"] = df_m_retorno["TELEFONO"].apply(lambda x: x.replace(".0",""))
temporal.shape

(664861, 37)

In [ ]:
dft_ct = temporal[temporal.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
dfm_str = df_m_retorno[df_m_retorno.TELEFONO == 'nan']
dfm_ct = df_m_retorno[df_m_retorno.TELEFONO != 'nan']
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()

# indx_= df_m_retorno.query("(NUMERO_DOCUMENTO == @x1) & (TELEFONO == @x2)").index
# indx = temporal.query("(NUMERO_DOCUMENTO == @y1) & (TELEFONO == @y2)").index

# pos_ = list()
# for i in range(1):
#   if (dft_ct.TELEFONO[dft_ct.NUMERO_DOCUMENTO==y1[0]]==y2[0]).iloc[0]:
#     pos_.append((dft_ct.TELEFONO[dft_ct.NUMERO_DOCUMENTO==y1[0]]==y2[0]).index[0])

# def fun1(x,y1,y2,i):
#   if (x.TELEFONO[x.NUMERO_DOCUMENTO==y1[i]]==y2[i]).iloc[0]:
#     return (x.TELEFONO[x.NUMERO_DOCUMENTO==y1[0]]==y2[0]).index[0]
# results = Parallel(n_jobs=2)(delayed(fun1)(dft_ct,y1,y2,i) for i in range(len(y1)))

# indices de sabana de retorno
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx2 = list(set(ind_doc) & set(ind_con))

# # indices de sabana actualizada
ind_doc = list(np.where(np.isin(y1 and x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2 and x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx_ = list(set(ind_doc) & set(ind_con))

In [ ]:
df_m_retorno.columns

Index(['NUMERO_DOCUMENTO', 'TELEFONO', 'Tipificacion', 'Venta', 'Gestionable'], dtype='object')

In [ ]:
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'Venta']]
temporal_.rename(columns={'Venta':'Emergia_Mayo_Venta'}, inplace= True)
# temporal_['indices'] = temporal_.index
# temporal_.head(5)
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'Venta']]
temporal_2.rename(columns={'Venta':'Emergia_Mayo_Venta'}, inplace= True)
# temporal_2['indices'] = temporal_.index
# temporal_2.head(5)
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Mayo_Venta_x', 'Emergia_Mayo_Venta_y']
temporal['Emergia_Mayo_Venta'] = 0
temporal['Emergia_Mayo_Venta'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'Gestionable']]
temporal_['Gestionable'] = 1
temporal_.rename(columns={'Gestionable':'Emergia_Mayo_Gestion'}, inplace= True)
# temporal_['indices'] = temporal_.index
# temporal_.head(5)
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'Gestionable']]
temporal_2['Gestionable'] = 1
temporal_2.rename(columns={'Gestionable':'Emergia_Mayo_Gestion'}, inplace= True)
# temporal_2['indices'] = temporal_.index
# temporal_2.head(5)
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Mayo_Gestion_x', 'Emergia_Mayo_Gestion_y']
temporal['Emergia_Mayo_Gestion'] = 0
temporal['Emergia_Mayo_Gestion'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1
# dfm_ct.Tipificacion.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [ ]:
dfm_ct['pos_con'] = 0
dfm_ct['pos_con'][dfm_ct.Tipificacion=='No contesta'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Contestador'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='ACW'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Teléfono no existe'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Incidencia CTI'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='No contesta - Agente'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Cuelga en espera'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Contestador - Agente'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='LLAMADA CORTADA'] = 1
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'pos_con']]
temporal_.rename(columns={'pos_con':'Emergia_Mayo_NoContacto'}, inplace= True)
# dfm_ct['pos_con'][dfm_ct.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

dfm_str['pos_con'] = 0
dfm_str['pos_con'][dfm_str.Tipificacion=='No contesta'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Contestador'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='ACW'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Teléfono no existe'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Incidencia CTI'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='No contesta - Agente'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Cuelga en espera'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Contestador - Agente'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='LLAMADA CORTADA'] = 1
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'pos_con']]
temporal_2.rename(columns={'pos_con':'Emergia_Mayo_NoContacto'}, inplace= True)
# dfm_str['pos_con'][dfm_str.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [ ]:
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Mayo_NoContacto_x', 'Emergia_Mayo_NoContacto_y']
temporal['Emergia_Mayo_NoContacto'] = 0
temporal['Emergia_Mayo_NoContacto'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# dfm_ct['pos_sin'] = 1
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='No contesta'] = 0
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='Contestador'] = 0
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='ACW'] = 0
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='Teléfono no existe'] = 0
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='Incidencia CTI'] = 0
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='No contesta - Agente'] = 0
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='Cuelga en espera'] = 0
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='Contestador - Agente'] = 0
# dfm_ct['pos_sin'][dfm_ct.Tipificacion=='LLAMADA CORTADA'] = 0
# temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'pos_sin']]
# temporal_.rename(columns={'pos_sin':'Emergia_Mayo_NoContacto'}, inplace= True)
# # dfm_ct['pos_con'][dfm_ct.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

# dfm_str['pos_sin'] = 1
# dfm_str['pos_sin'][dfm_str.Tipificacion=='No contesta'] = 0
# dfm_str['pos_sin'][dfm_str.Tipificacion=='Contestador'] = 0
# dfm_str['pos_sin'][dfm_str.Tipificacion=='ACW'] = 0
# dfm_str['pos_sin'][dfm_str.Tipificacion=='Teléfono no existe'] = 0
# dfm_str['pos_sin'][dfm_str.Tipificacion=='Incidencia CTI'] = 0
# dfm_str['pos_sin'][dfm_str.Tipificacion=='No contesta - Agente'] = 0
# dfm_str['pos_sin'][dfm_str.Tipificacion=='Cuelga en espera'] = 0
# dfm_str['pos_sin'][dfm_str.Tipificacion=='Contestador - Agente'] = 0
# dfm_str['pos_sin'][dfm_str.Tipificacion=='LLAMADA CORTADA'] = 0
# temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'pos_sin']]
# temporal_2.rename(columns={'pos_sin':'Emergia_Mayo_Contacto'}, inplace= True)
# # dfm_str['pos_con'][dfm_str.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

In [ ]:
# temporal1 = temporal.copy()
# temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
# temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# # temporal_1.head(1)
# temp = ['Emergia_Mayo_Contacto_x', 'Emergia_Mayo_Contacto_y']
# temporal['Emergia_Mayo_Contacto'] = 0
# temporal['Emergia_Mayo_Contacto'].iloc[temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

In [ ]:
temporal.columns

Index(['TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NOMBRE_CLIENTE', 'TELEFONO',
       'TELEFONO_CONTACTO_1', 'TELEFONO_CONTACTO_2', 'TELEFONO_CONTACTO_3',
       'LOCALIDAD', 'BARRIO', 'DIRECCION_INSTALACION', 'LATITUD', 'LONGITUD',
       'PAQUETE', 'PAQUETE_PLAN', 'CICLO', 'ESTADO', 'HABEAS_DATA',
       'MOVIL_ETB_ACTUAL', 'MOVIL_ETB_HISTORICO', 'Tipo_oferta',
       'Antiguedad_meses', 'VELOCIDAD', 'TECNOLOGIA', 'TARIFA_CONFIGURADA',
       'ESTRATO_USO', 'celular_1', 'celular_2', 'celular_3', 'C_SEG_UEN',
       'Localidad_zona', 'tipo_estrato', 'Tipo_velocidad', 'grupos', 'pqr',
       'TELEFONO2', 'Emergia_Mayo_nuevo', 'Emergia_Mayo_Bajas',
       'Emergia_Mayo_Venta', 'Emergia_Mayo_Gestion',
       'Emergia_Mayo_NoContacto'],
      dtype='object')

In [ ]:
temporal.Emergia_Mayo_NoContacto.sum()

66544

In [ ]:
#@title cantidad de ragistros con los nuevos mes mayo con NaNs
dft_ct.shape[0]+dft_st.shape[0]+dfm_st.shape[0]

664861

In [ ]:
dt_total_Mayo = temporal.copy()
print('Registros Actualizados: {}'.format(dt_total_Mayo.shape))

Registros Actualizados: (664861, 40)


In [ ]:
dt_total_Mayo = temporal.copy()
print('Registros Actualizados: {}'.format(dt_total_Mayo.shape))

Registros Actualizados: (664861, 40)


In [ ]:
dt_total_Mayo.head(1)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,tipo_estrato,Tipo_velocidad,grupos,pqr,TELEFONO2,Emergia_Mayo_nuevo,Emergia_Mayo_Bajas,Emergia_Mayo_Venta,Emergia_Mayo_Gestion,Emergia_Mayo_NoContacto
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,medio,Menor 100,Otros,0,13561955,0,NaN,0,1,0


In [ ]:
# dt_total_Mayo.to_csv(path+'seguimiento_corte_Mayo.csv')

In [ ]:
# dt_total_Mayo = pd.read_csv(path+'seguimiento_corte_Mayo.csv')##

In [ ]:
# # dt_total_Mayo.TELEFONO = dt_total_Mayo.TELEFONO.astype(float)
# # dt_total_Mayo.TELEFONO[dt_total_Mayo.TELEFONO.notna()] = dt_total_Mayo.TELEFONO[dt_total_Mayo.TELEFONO.notna()].astype(int)
# dt_total_Mayo.TELEFONO = dt_total_Mayo.TELEFONO.astype(str)
# dt_total_Mayo["TELEFONO"] = dt_total_Mayo["TELEFONO"].apply(lambda x: x.replace(".0",""))
# dt_total_Mayo["TELEFONO"] = dt_total_Mayo["TELEFONO"].apply(lambda x: x.replace("SR-",""))
# dt_total_Mayo["TELEFONO"] = dt_total_Mayo["TELEFONO"].apply(lambda x: x.replace("ccc","99999"))
# # dt_total_Mayo.TELEFONO = dt_total_Mayo.TELEFONO.astype(float)
# # dt_total_Mayo.TELEFONO[dt_total_Mayo.TELEFONO.notna()] = dt_total_Mayo.TELEFONO[dt_total_Mayo.TELEFONO.notna()].astype(int)
# # dt_total_Mayo.TELEFONO = dt_total_Mayo.TELEFONO.astype(str)

In [ ]:
df_t_ = dt_total_Mayo.copy()

In [ ]:
df_t_.shape

(664861, 40)

In [ ]:
df_t_.Emergia_Mayo_Bajas.sum()

22037.0

In [ ]:
# df_t_[df_t_.TELEFONO != 'nan'][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Mayo_nuevo!=1].shape

# Junio

In [ ]:
#@title ###Seleccionar mes para analizar {run: "auto", vertical-output: true }
Mes = 'Junio' 
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['Base Total de Clientes de la Operacion Fija_20220629.xlsx',
 'Retorno_Junio.xlsx',
 'Datos_seguimiento_Junio.csv',
 'seguimiento_corte_Junio.csv']

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Base Total' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636435 entries, 0 to 636434
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   TIPO_DOCUMENTO         636435 non-null  object 
 1   NUMERO_DOCUMENTO       636435 non-null  object 
 2   NOMBRE_CLIENTE         635786 non-null  object 
 3   TELEFONO               619837 non-null  object 
 4   TELEFONO_CONTACTO_1    490069 non-null  float64
 5   TELEFONO_CONTACTO_2    533453 non-null  float64
 6   TELEFONO_CONTACTO_3    335932 non-null  float64
 7   LOCALIDAD              565275 non-null  object 
 8   BARRIO                 548881 non-null  object 
 9   DIRECCION_INSTALACION  636322 non-null  object 
 10  LATITUD                568722 non-null  float64
 11  LONGITUD               568722 non-null  float64
 12  PAQUETE                636435 non-null  object 
 13  PAQUETE_PLAN           601862 non-null  object 
 14  CICLO                  634324 non-nu

In [ ]:
df_m.shape

(636435, 34)

In [ ]:
df_m.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m.reset_index(drop=True, inplace=True)
df_m.shape

(636430, 34)

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
# data con mes Junio
dft_st = df_t_[df_t_.TELEFONO.isna()]
dft_ct = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc),len(ind_con),len(indx))
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Registros nuevos: (24844, 34)


In [ ]:
dfm_st[[df_t_.columns[dfm_ct.columns.shape[0]:].values, 'Emergia_Junio_nuevo']] = 0
df_New[[df_t_.columns[df_New.columns.shape[0]:].values, 'Emergia_Junio_nuevo']] = 0
df_New['Emergia_Junio_nuevo'] = 1
df_t_['Emergia_Junio_nuevo'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [ ]:
#@title cantidad de ragistros con los nuevos mes mayo sin NaNs
dft_ct.shape[0]+df_New.shape[0]

645571

In [ ]:
temporal = pd.DataFrame()
temporal = pd.concat([df_t_, df_New, dfm_st], axis=0)
temporal.shape

(706302, 41)

In [ ]:
temporal.reset_index(drop = True, inplace= True)

In [ ]:
dft_ct = temporal[temporal.TELEFONO.notna()][temporal.Emergia_Mayo_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
df_Eli = dft_ct.drop(indx,axis=0)
print('Registros eliminados: {}'.format(df_Eli.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Registros eliminados: (25683, 41)


In [ ]:
df_Eli = df_Eli[['NUMERO_DOCUMENTO', 'TELEFONO']]
df_Eli['Emergia_Junio_Bajas'] = 1

In [ ]:
temporal = temporal.merge(df_Eli, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')

In [ ]:
# temporal[temporal.Emergia_Mayo_nuevo!=1][temporal.Emergia_Junio_nuevo!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1].shape

In [ ]:
# temporal[temporal.Emergia_Junio_Bajas!=1].shape

In [ ]:
# temporal[temporal.Emergia_Mayo_nuevo!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_nuevo!=1][temporal.Emergia_Junio_Bajas!=1].shape

In [ ]:
temporal['Emergia_Junio_Bajas'].sum()

25683.0

In [ ]:
#@title ###Carga del mes selecionado en retorno { run: "auto", vertical-output: true }
Nombre_archivo = 'Retorno' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m_retorno = pd.read_excel(path+nombre_completo)
df_m_retorno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216958 entries, 0 to 216957
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   NUMERO_DOCUMENTO_CLIENTE  216958 non-null  object 
 1   NUMERO_CONEXION           201598 non-null  float64
 2   Tipificacion              214991 non-null  object 
 3   Venta                     216958 non-null  float64
 4   Gestionable               216958 non-null  float64
dtypes: float64(3), object(2)
memory usage: 8.3+ MB


In [ ]:
df_m_retorno.rename(columns={'NUMERO_CONEXION':'TELEFONO','NUMERO_DOCUMENTO_CLIENTE':'NUMERO_DOCUMENTO'}, inplace= True)
df_m_retorno.TELEFONO.shape

(216958,)

In [ ]:
df_m_retorno['TELEFONO'] = df_m_retorno['TELEFONO'].astype(str)
df_m_retorno["TELEFONO"] = df_m_retorno["TELEFONO"].apply(lambda x: x.replace(".0",""))
df_m_retorno.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='last', inplace=True)
df_m_retorno.TELEFONO.shape

(216832,)

In [ ]:
dft_ct = temporal[temporal.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
dfm_str = df_m_retorno[df_m_retorno.TELEFONO.isna()]
dfm_ct = df_m_retorno[df_m_retorno.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()

# indx = df_tot.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# indx_= df_m_retorno.query("NUMERO_DOCUMENTO == @x1 and TELEFONO == @x2").index

# indices de sabana de retorno
# ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
# ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
# indx = list(set(ind_doc) & set(ind_con))

# # indices de sabana actualizada
# ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
# ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
# indx_ = list(set(ind_doc) & set(ind_con))

In [ ]:
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'Venta']]
temporal_.rename(columns={'Venta':'Emergia_Junio_Venta'}, inplace= True)
# temporal_['indices'] = temporal_.index
# temporal_.head(5)
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'Venta']]
temporal_2.rename(columns={'Venta':'Emergia_Junio_Venta'}, inplace= True)
# temporal_2['indices'] = temporal_.index
# temporal_2.head(5)
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Junio_Venta_x', 'Emergia_Junio_Venta_y']
temporal['Emergia_Junio_Venta'] = 0
temporal['Emergia_Junio_Venta'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'Gestionable']]
temporal_['Gestionable'] = 1
temporal_.rename(columns={'Gestionable':'Emergia_Junio_Gestion'}, inplace= True)
# temporal_['indices'] = temporal_.index
# temporal_.head(5)
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'Gestionable']]
temporal_2['Gestionable'] = 1
temporal_2.rename(columns={'Gestionable':'Emergia_Junio_Gestion'}, inplace= True)
# temporal_2['indices'] = temporal_.index
# temporal_2.head(5)
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Junio_Gestion_x', 'Emergia_Junio_Gestion_y']
temporal['Emergia_Junio_Gestion'] = 0
temporal['Emergia_Junio_Gestion'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1
# dfm_ct.Tipificacion.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dfm_ct['pos_con'] = 0
dfm_ct['pos_con'][dfm_ct.Tipificacion=='No contesta'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Contestador'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='ACW'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Teléfono no existe'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Incidencia CTI'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='No contesta - Agente'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Cuelga en espera'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Contestador - Agente'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='LLAMADA CORTADA'] = 1
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'pos_con']]
temporal_.rename(columns={'pos_con':'Emergia_Junio_NoContacto'}, inplace= True)
# dfm_ct['pos_con'][dfm_ct.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

dfm_str['pos_con'] = 0
dfm_str['pos_con'][dfm_str.Tipificacion=='No contesta'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Contestador'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='ACW'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Teléfono no existe'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Incidencia CTI'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='No contesta - Agente'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Cuelga en espera'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Contestador - Agente'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='LLAMADA CORTADA'] = 1
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'pos_con']]
temporal_2.rename(columns={'pos_con':'Emergia_Junio_NoContacto'}, inplace= True)
# dfm_str['pos_con'][dfm_str.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [ ]:
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Junio_NoContacto_x', 'Emergia_Junio_NoContacto_y']
temporal['Emergia_Junio_NoContacto'] = 0
temporal['Emergia_Junio_NoContacto'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# temporal.columns

In [ ]:
# temporal.shape

In [ ]:
temporal.Emergia_Junio_Bajas.sum()

25683.0

In [ ]:
temporal.head(2)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,Emergia_Mayo_nuevo,Emergia_Mayo_Bajas,Emergia_Mayo_Venta,Emergia_Mayo_Gestion,Emergia_Mayo_NoContacto,Emergia_Junio_nuevo,Emergia_Junio_Bajas,Emergia_Junio_Venta,Emergia_Junio_Gestion,Emergia_Junio_NoContacto
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,0,NaN,0,1,0,0,NaN,0,0,0
1,CC,1000719025,NATALIA MONTENEGRO LOPEZ,17659219,6.015875e+09,3.115875e+09,NaN,CIUDAD BOLIVAR,RAFAEL ESCAMILLA,CL 60A SUR 68 08 TO 3 AP 205,...,0,NaN,0,0,0,0,NaN,0,0,0


In [ ]:
dt_total_Junio = temporal.copy()
print('Registros Actualizados: {}'.format(dt_total_Junio.shape))

Registros Actualizados: (706302, 45)


In [ ]:
# dt_total_Junio.to_csv(path+'seguimiento_corte_Junio.csv')

In [ ]:
dt_total_Junio.head(1)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,Emergia_Mayo_nuevo,Emergia_Mayo_Bajas,Emergia_Mayo_Venta,Emergia_Mayo_Gestion,Emergia_Mayo_NoContacto,Emergia_Junio_nuevo,Emergia_Junio_Bajas,Emergia_Junio_Venta,Emergia_Junio_Gestion,Emergia_Junio_NoContacto
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,0,NaN,0,1,0,0,NaN,0,0,0


In [ ]:
df_t_ = dt_total_Junio.copy()

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
# df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1].shape

In [ ]:
# df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Mayo_nuevo!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Junio_nuevo!=1].shape

# Julio

In [ ]:
#@title ###Seleccionar mes para analizar {run: "auto", vertical-output: true }
Mes = 'Julio' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['Base Total de Clientes de la Operacion Fija_20220728.xlsx',
 'Retorno_Julio.xlsx',
 'seguimiento_corte_Julio.csv']

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Base Total' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636634 entries, 0 to 636633
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   TIPO_DOCUMENTO         636634 non-null  object 
 1   NUMERO_DOCUMENTO       636634 non-null  object 
 2   NOMBRE_CLIENTE         636021 non-null  object 
 3   TELEFONO               615394 non-null  object 
 4   TELEFONO_CONTACTO_1    492798 non-null  float64
 5   TELEFONO_CONTACTO_2    538401 non-null  float64
 6   TELEFONO_CONTACTO_3    356182 non-null  float64
 7   LOCALIDAD              567605 non-null  object 
 8   BARRIO                 550200 non-null  object 
 9   DIRECCION_INSTALACION  636550 non-null  object 
 10  LATITUD                570167 non-null  float64
 11  LONGITUD               570167 non-null  float64
 12  PAQUETE                636634 non-null  object 
 13  PAQUETE_PLAN           603996 non-null  object 
 14  CICLO                  634755 non-nu

In [ ]:
df_m.shape

(636634, 35)

In [ ]:
df_m.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m.reset_index(drop=True, inplace=True)
df_m.shape

(636625, 35)

In [ ]:
df_t_.shape

(706302, 45)

In [ ]:
df_t_.reset_index(drop=True, inplace=True)


In [ ]:
# data con mes Julio
dft_st = df_t_[df_t_.TELEFONO.isna()]
dft_ct = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc),len(ind_con),len(indx))
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Registros nuevos: (18857, 35)


In [ ]:
dfm_st[[df_t_.columns[dfm_ct.columns.shape[0]:].values, 'Emergia_Julio_nuevo']] = 0
df_New[[df_t_.columns[df_New.columns.shape[0]:].values, 'Emergia_Julio_nuevo']] = 0
df_New['Emergia_Julio_nuevo'] = 1
df_t_['Emergia_Julio_nuevo'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [ ]:
temporal = pd.DataFrame()
temporal = pd.concat([df_t_, df_New, dfm_st], axis=0)
temporal.reset_index(drop=True, inplace=True)
temporal.shape

(746397, 47)

In [ ]:
dft_st = pd.concat([temporal[temporal.TELEFONO.isna()],temporal[temporal.Emergia_Mayo_Bajas==1],
                    temporal[temporal.Emergia_Junio_Bajas==1]], axis=0)
dft_ct = temporal[temporal.TELEFONO.notna()][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# datos = dft_ct[['TELEFONO','NUMERO_DOCUMENTO']].iloc[indx]
# y1 = datos['NUMERO_DOCUMENTO'].tolist()
# y2 = datos['TELEFONO'].tolist()
# indx = df_t.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
df_Eli = dft_ct.drop(indx,axis=0)
print('Registros eliminados: {}'.format(df_Eli.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Registros eliminados: (23194, 47)


In [ ]:
dft_ct['Emergia_Julio_Bajas'] = 0
dft_ct['Emergia_Julio_Bajas'].iloc[df_Eli.index] = 1
dft_st['Emergia_Julio_Bajas'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
temporal = pd.concat([dft_ct, dft_st], axis=0)
temporal.shape

(746397, 48)

In [ ]:
temporal['Emergia_Julio_Bajas'].sum()

23194

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Retorno' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m_retorno = pd.read_excel(path+nombre_completo)
df_m_retorno.info()

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146836 entries, 0 to 146835
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   NUMERO_DOCUMENTO_CLIENTE  146836 non-null  object 
 1   NUMERO_CONEXION           135918 non-null  float64
 2   Tipificacion              146835 non-null  object 
 3   Venta                     146836 non-null  float64
 4   Gestionable               146836 non-null  float64
dtypes: float64(3), object(2)
memory usage: 5.6+ MB


In [ ]:
df_m_retorno.rename(columns={'NUMERO_CONEXION':'TELEFONO','NUMERO_DOCUMENTO_CLIENTE':'NUMERO_DOCUMENTO'}, inplace= True)
df_m_retorno.TELEFONO.shape

(146836,)

In [ ]:
df_m_retorno['TELEFONO'] = df_m_retorno['TELEFONO'].astype(str)
df_m_retorno["TELEFONO"] = df_m_retorno["TELEFONO"].apply(lambda x: x.replace(".0",""))
df_m_retorno.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='last', inplace=True)
df_m_retorno.TELEFONO.shape

(146216,)

In [ ]:
dft_ct = temporal[temporal.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
dfm_str = df_m_retorno[df_m_retorno.TELEFONO.isna()]
dfm_ct = df_m_retorno[df_m_retorno.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()

# indx = df_tot.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# indx_= df_m_retorno.query("NUMERO_DOCUMENTO == @x1 and TELEFONO == @x2").index

# indices de sabana de retorno
# ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
# ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
# indx = list(set(ind_doc) & set(ind_con))

# # indices de sabana actualizada
# ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
# ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
# indx_ = list(set(ind_doc) & set(ind_con))


In [ ]:
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'Venta']]
temporal_.rename(columns={'Venta':'Emergia_Julio_Venta'}, inplace= True)
# temporal_['indices'] = temporal_.index
# temporal_.head(5)
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'Venta']]
temporal_2.rename(columns={'Venta':'Emergia_Julio_Venta'}, inplace= True)
# temporal_2['indices'] = temporal_.index
# temporal_2.head(5)
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Julio_Venta_x', 'Emergia_Julio_Venta_y']
temporal['Emergia_Julio_Venta'] = 0
temporal['Emergia_Julio_Venta'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'Gestionable']]
temporal_['Gestionable'] = 1
temporal_.rename(columns={'Gestionable':'Emergia_Julio_Gestion'}, inplace= True)
# temporal_['indices'] = temporal_.index
# temporal_.head(5)
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'Gestionable']]
temporal_2['Gestionable'] = 1
temporal_2.rename(columns={'Gestionable':'Emergia_Julio_Gestion'}, inplace= True)
# temporal_2['indices'] = temporal_.index
# temporal_2.head(5)
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Julio_Gestion_x', 'Emergia_Julio_Gestion_y']
temporal['Emergia_Julio_Gestion'] = 0
temporal['Emergia_Julio_Gestion'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1
# dfm_ct.Tipificacion.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dfm_ct['pos_con'] = 0
dfm_ct['pos_con'][dfm_ct.Tipificacion=='No contesta'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Contestador'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='ACW'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Teléfono no existe'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Incidencia CTI'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='No contesta - Agente'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Cuelga en espera'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Contestador - Agente'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='LLAMADA CORTADA'] = 1
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'pos_con']]
temporal_.rename(columns={'pos_con':'Emergia_Julio_NoContacto'}, inplace= True)
# dfm_ct['pos_con'][dfm_ct.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

dfm_str['pos_con'] = 0
dfm_str['pos_con'][dfm_str.Tipificacion=='No contesta'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Contestador'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='ACW'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Teléfono no existe'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Incidencia CTI'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='No contesta - Agente'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Cuelga en espera'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Contestador - Agente'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='LLAMADA CORTADA'] = 1
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'pos_con']]
temporal_2.rename(columns={'pos_con':'Emergia_Julio_NoContacto'}, inplace= True)
# dfm_str['pos_con'][dfm_str.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [ ]:
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Julio_NoContacto_x', 'Emergia_Julio_NoContacto_y']
temporal['Emergia_Julio_NoContacto'] = 0
temporal['Emergia_Julio_NoContacto'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# temporal.columns

In [ ]:
# temporal.shape

In [ ]:
temporal.Emergia_Julio_Bajas.sum()

23194

In [ ]:
temporal.head(2)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,Emergia_Junio_Bajas,Emergia_Junio_Venta,Emergia_Junio_Gestion,Emergia_Junio_NoContacto,Emergia_Julio_nuevo,CORREO_ELECTRONICO,Emergia_Julio_Bajas,Emergia_Julio_Venta,Emergia_Julio_Gestion,Emergia_Julio_NoContacto
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,NaN,0,0,0,0,NaN,0,0,0,0
1,CC,1000719025,NATALIA MONTENEGRO LOPEZ,17659219,6.015875e+09,3.115875e+09,NaN,CIUDAD BOLIVAR,RAFAEL ESCAMILLA,CL 60A SUR 68 08 TO 3 AP 205,...,NaN,0,0,0,0,NaN,0,0,0,0


In [ ]:
dt_total_Julio = temporal.copy()
print('Registros Actualizados: {}'.format(dt_total_Julio.shape))

Registros Actualizados: (746397, 51)


In [ ]:
# dt_total_Julio.to_csv(path+'seguimiento_corte_Julio.csv')

In [ ]:
dt_total_Julio.head(1)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,Emergia_Junio_Bajas,Emergia_Junio_Venta,Emergia_Junio_Gestion,Emergia_Junio_NoContacto,Emergia_Julio_nuevo,CORREO_ELECTRONICO,Emergia_Julio_Bajas,Emergia_Julio_Venta,Emergia_Julio_Gestion,Emergia_Julio_NoContacto
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,NaN,0,0,0,0,NaN,0,0,0,0


In [ ]:
# dt_total_Julio.Emergia_Julio_Bajas.sum()

In [ ]:
df_t_ = dt_total_Julio.copy()

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

# Agosto

In [ ]:
#@title ###Seleccionar mes para analizar {run: "auto", vertical-output: true }
Mes = 'Agosto' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['ETB_TOTAL_29k_AGOSTO.xlsx',
 'Base Total de Clientes de la Operacion Fija_20220829.xlsx',
 'Base Total de Clientes de la Operacion Fija_20220829_New_19K_normalizada.xlsx',
 'Base Total de Clientes de la Operacion Fija_20220829_actualizada_613K_normalizada.xlsx',
 'Majorel_82k.xlsx',
 'ETB_Total_Seguimiento1_agosto_638k_normalizada.xlsx',
 'ETB_Total_Seguimiento2_agosto_638k_normalizada.xlsx',
 'Retorno_Agosto.xlsx',
 'ETB_Total_393k_sin_agosto_normalizada.xlsx',
 'ETB_Total3_393k_sin_agosto_normalizada_cruce_ventas.xlsx',
 'ETB_Total_model_393k_sin_agosto_normalizada_cruce_ventas.xlsx',
 'Prediccion_ETB_Total_model_393k_sin_agosto_normalizada_cruce_ventas.xlsx',
 'Prediccion_ETB_Total_model_393k_sin_agosto_normalizada_cruce_ventas_v2_depurado.xlsx',
 'Base Total Actualizada de Clientes de la Operacion_fija_20220428_620K_normalizada_619K_normalizada_615K_normalizada_612K_normalizada.xlsx',
 'Base Total de Clientes de la Operacion Fija_20220829_New_612K_normalizada.xlsx',
 'Majorel_14k

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Base Total de Clientes de la Operacion Fija_20220829.xlsx' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638238 entries, 0 to 638237
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   TIPO_DOCUMENTO         638238 non-null  object        
 1   NUMERO_DOCUMENTO       638238 non-null  object        
 2   NOMBRE_CLIENTE         637566 non-null  object        
 3   TELEFONO               612884 non-null  object        
 4   TELEFONO_CONTACTO_1    495229 non-null  float64       
 5   TELEFONO_CONTACTO_2    544550 non-null  float64       
 6   TELEFONO_CONTACTO_3    371007 non-null  float64       
 7   LOCALIDAD              574598 non-null  object        
 8   BARRIO                 556677 non-null  object        
 9   DIRECCION_INSTALACION  638147 non-null  object        
 10  LATITUD                573321 non-null  float64       
 11  LONGITUD               573321 non-null  float64       
 12  PAQUETE                638238 non-null  obje

In [ ]:
df_m.shape

(638238, 37)

In [ ]:
df_m.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m.reset_index(drop=True, inplace=True)
df_m.shape

(638233, 37)

In [ ]:
df_t_.shape

(746397, 51)

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
# data con mes Julio
dft_st = df_t_[df_t_.TELEFONO.isna()]
dft_ct = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc),len(ind_con),len(indx))
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Registros nuevos: (19461, 37)


In [ ]:
dfm_st[[df_t_.columns[dfm_ct.columns.shape[0]:].values, 'Emergia_Agosto_nuevo']] = 0
df_New[[df_t_.columns[df_New.columns.shape[0]:].values, 'Emergia_Agosto_nuevo']] = 0
df_New['Emergia_Agosto_nuevo'] = 1
df_t_['Emergia_Agosto_nuevo'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [ ]:
temporal = pd.DataFrame()
temporal = pd.concat([df_t_, df_New, dfm_st], axis=0)
temporal.reset_index(drop=True, inplace=True)
temporal.shape

(791210, 54)

In [ ]:
temporal.Emergia_Julio_Bajas.sum()

23194

In [ ]:
dft_st = pd.concat([temporal[temporal.TELEFONO.isna()],temporal[temporal.Emergia_Mayo_Bajas==1],
                    temporal[temporal.Emergia_Junio_Bajas==1], temporal[temporal.Emergia_Julio_Bajas==1]], axis=0)
dft_ct = temporal[temporal.TELEFONO.notna()][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# datos = dft_ct[['TELEFONO','NUMERO_DOCUMENTO']].iloc[indx]
# y1 = datos['NUMERO_DOCUMENTO'].tolist()
# y2 = datos['TELEFONO'].tolist()
# indx = df_t.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
df_Eli = dft_ct.drop(indx,axis=0)
print('Registros eliminados: {}'.format(df_Eli.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Registros eliminados: (21883, 54)


In [ ]:
dft_ct['Emergia_Agosto_Bajas'] = 0
dft_ct['Emergia_Agosto_Bajas'].iloc[df_Eli.index] = 1
dft_st['Emergia_Agosto_Bajas'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
temporal = pd.concat([dft_ct, dft_st], axis=0)
temporal.shape

(791210, 55)

In [ ]:
temporal['Emergia_Agosto_Bajas'].sum()

21883

In [ ]:
#@title ###Carga del retorno del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Retorno' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m_retorno = pd.read_excel(path+nombre_completo)
df_m_retorno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244697 entries, 0 to 244696
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   NUMERO_DOCUMENTO_CLIENTE  244697 non-null  object 
 1   NUMERO_CONEXION           234822 non-null  float64
 2   Tipificacion              244423 non-null  object 
 3   Venta                     244697 non-null  float64
 4   Gestionable               244697 non-null  float64
dtypes: float64(3), object(2)
memory usage: 9.3+ MB


In [ ]:
df_m_retorno.rename(columns={'NUMERO_CONEXION':'TELEFONO','NUMERO_DOCUMENTO_CLIENTE':'NUMERO_DOCUMENTO'}, inplace= True)
df_m_retorno.TELEFONO.shape

(244697,)

In [ ]:
df_m_retorno['TELEFONO'] = df_m_retorno['TELEFONO'].astype(str)
df_m_retorno["TELEFONO"] = df_m_retorno["TELEFONO"].apply(lambda x: x.replace(".0",""))
df_m_retorno.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='last', inplace=True)
df_m_retorno.TELEFONO.shape

(230207,)

In [ ]:
dft_ct = temporal[temporal.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
dfm_str = df_m_retorno[df_m_retorno.TELEFONO.isna()]
dfm_ct = df_m_retorno[df_m_retorno.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()

# indx = df_tot.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# indx_= df_m_retorno.query("NUMERO_DOCUMENTO == @x1 and TELEFONO == @x2").index

# indices de sabana de retorno
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))

# # indices de sabana actualizada
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx_ = list(set(ind_doc) & set(ind_con))

In [ ]:
df_m_retorno.Venta

203       0.0
204       0.0
205       0.0
206       0.0
207       0.0
         ... 
244692    0.0
244693    0.0
244694    0.0
244695    0.0
244696    0.0
Name: Venta, Length: 230207, dtype: float64

In [ ]:
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'Venta']]
temporal_.rename(columns={'Venta':'Emergia_Agosto_Venta'}, inplace= True)
# temporal_['indices'] = temporal_.index
# temporal_.head(5)
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'Venta']]
temporal_2.rename(columns={'Venta':'Emergia_Agosto_Venta'}, inplace= True)
# temporal_2['indices'] = temporal_.index
# temporal_2.head(5)
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Agosto_Venta_x', 'Emergia_Agosto_Venta_y']
temporal['Emergia_Agosto_Venta'] = 0
temporal['Emergia_Agosto_Venta'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'Gestionable']]
temporal_['Gestionable'] = 1
temporal_.rename(columns={'Gestionable':'Emergia_Agosto_Gestion'}, inplace= True)
# temporal_['indices'] = temporal_.index
# temporal_.head(5)
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'Gestionable']]
temporal_2['Gestionable'] = 1
temporal_2.rename(columns={'Gestionable':'Emergia_Agosto_Gestion'}, inplace= True)
# temporal_2['indices'] = temporal_.index
# temporal_2.head(5)
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Agosto_Gestion_x', 'Emergia_Agosto_Gestion_y']
temporal['Emergia_Agosto_Gestion'] = 0
temporal['Emergia_Agosto_Gestion'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1
# dfm_ct.Tipificacion.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dfm_ct['pos_con'] = 0
dfm_ct['pos_con'][dfm_ct.Tipificacion=='No contesta'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Contestador'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='ACW'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Teléfono no existe'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Incidencia CTI'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='No contesta - Agente'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Cuelga en espera'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='Contestador - Agente'] = 1
dfm_ct['pos_con'][dfm_ct.Tipificacion=='LLAMADA CORTADA'] = 1
temporal_ = dfm_ct[['NUMERO_DOCUMENTO', 'TELEFONO', 'pos_con']]
temporal_.rename(columns={'pos_con':'Emergia_Agosto_NoContacto'}, inplace= True)
# dfm_ct['pos_con'][dfm_ct.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

dfm_str['pos_con'] = 0
dfm_str['pos_con'][dfm_str.Tipificacion=='No contesta'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Contestador'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='ACW'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Teléfono no existe'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Incidencia CTI'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='No contesta - Agente'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Cuelga en espera'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='Contestador - Agente'] = 1
dfm_str['pos_con'][dfm_str.Tipificacion=='LLAMADA CORTADA'] = 1
temporal_2 = dfm_str[['NUMERO_DOCUMENTO', 'pos_con']]
temporal_2.rename(columns={'pos_con':'Emergia_Agosto_NoContacto'}, inplace= True)
# dfm_str['pos_con'][dfm_str.Tipificacion=='Ilocalizable por horarios'] = 1 # preguntar

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [ ]:
temporal1 = temporal.copy()
temporal_1 = temporal1.merge(temporal_, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')
temporal_1 = temporal_1.merge(temporal_2, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')
# temporal_1.head(1)
temp = ['Emergia_Agosto_NoContacto_x', 'Emergia_Agosto_NoContacto_y']
temporal['Emergia_Agosto_NoContacto'] = 0
temporal['Emergia_Agosto_NoContacto'][temporal_1[temp].fillna(0).sum(axis=1)>0] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# temporal.columns

In [ ]:
temporal.shape

(791210, 58)

In [ ]:
# temporal.head(2)

In [ ]:
dt_total_Agosto = temporal.copy()
print('Registros Actualizados: {}'.format(dt_total_Agosto.shape))

Registros Actualizados: (791210, 58)


In [ ]:
# dt_total_Agosto.to_csv(path+'seguimiento_corte_Agosto.csv')

In [ ]:
# dt_total_Agosto = pd.read_csv(path+'seguimiento_corte_Agosto.csv')

In [ ]:
dt_total_Agosto.head(1)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,Emergia_Julio_Venta,Emergia_Julio_Gestion,Emergia_Julio_NoContacto,Emergia_Agosto_nuevo,FECHA_INICIO,FUENTE,Emergia_Agosto_Bajas,Emergia_Agosto_Venta,Emergia_Agosto_Gestion,Emergia_Agosto_NoContacto
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,0,0,0,0,NaT,NaN,0,0,1,0


In [ ]:
dt_total_Agosto.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df_t_ = dt_total_Agosto.copy()

In [ ]:
# df_t_.columns

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

# Septiembre

In [ ]:
#@title ###Seleccionar mes para analizar {run: "auto", vertical-output: true }
Mes = 'Septiembre' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['Base Total de Clientes de la OperaciÃ³n Fija_20220928.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20220928_Nuevos_19k_normalizada.xls',
 'Seguimiento_global_ETB_corte_Septiembre.csv',
 'seguimiento_corte_Septiembre.csv',
 'Majorel_Septiembre.xlsx',
 'Etb_total_septiembre.xlsx',
 'Retorno Septiembre.xlsx']

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20220928.xlsx' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

In [ ]:
df_m.shape

(639911, 39)

In [ ]:
df_m.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m.reset_index(drop=True, inplace=True)
df_m.shape

(639640, 39)

In [ ]:
df_t_.shape

(791210, 58)

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
# data con mes Septiembre
dft_st = df_t_[df_t_.TELEFONO.isna()]
dft_ct = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc),len(ind_con),len(indx))
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Registros nuevos: (19544, 39)


In [ ]:
dfm_st[[df_t_.columns[dfm_ct.columns.shape[0]:].values, 'Emergia_Septiembre_nuevo']] = 0
df_New[[df_t_.columns[df_New.columns.shape[0]:].values, 'Emergia_Septiembre_nuevo']] = 0
df_New['Emergia_Septiembre_nuevo'] = 1
df_t_['Emergia_Septiembre_nuevo'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [ ]:
temporal = pd.DataFrame()
temporal = pd.concat([df_t_, df_New, dfm_st], axis=0)
temporal.reset_index(drop=True, inplace=True)
temporal.shape

(840817, 61)

In [ ]:
dft_st = pd.concat([temporal[temporal.TELEFONO.isna()],temporal[temporal.Emergia_Mayo_Bajas==1],
                    temporal[temporal.Emergia_Junio_Bajas==1], temporal[temporal.Emergia_Julio_Bajas==1], 
                    temporal[temporal.Emergia_Agosto_Bajas==1]], axis=0)
dft_ct = temporal[temporal.TELEFONO.notna()][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# datos = dft_ct[['TELEFONO','NUMERO_DOCUMENTO']].iloc[indx]
# y1 = datos['NUMERO_DOCUMENTO'].tolist()
# y2 = datos['TELEFONO'].tolist()
# indx = df_t.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
df_Eli = dft_ct.drop(indx,axis=0)
print('Registros eliminados: {}'.format(df_Eli.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Registros eliminados: (22808, 61)


In [ ]:
dft_ct['Emergia_Septiembre_Bajas'] = 0
dft_ct['Emergia_Septiembre_Bajas'].iloc[df_Eli.index] = 1
dft_st['Emergia_Septiembre_Bajas'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
temporal = pd.concat([dft_ct, dft_st], axis=0)
temporal.shape

(840817, 62)

In [ ]:
temporal['Emergia_Septiembre_Bajas'].sum()

22808

In [ ]:
temporal.head(2)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,FECHA_INICIO,FUENTE,Emergia_Agosto_Bajas,Emergia_Agosto_Venta,Emergia_Agosto_Gestion,Emergia_Agosto_NoContacto,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,NaT,NaN,0,0,1,0,0,NaN,NaN,0
1,CC,1000719025,NATALIA MONTENEGRO LOPEZ,17659219,6.015875e+09,3.115875e+09,NaN,CIUDAD BOLIVAR,RAFAEL ESCAMILLA,CL 60A SUR 68 08 TO 3 AP 205,...,NaT,NaN,0,0,0,0,0,NaN,NaN,0


In [ ]:
dt_total_Septiembre = temporal.copy()
print('Registros Actualizados: {}'.format(dt_total_Septiembre.shape))

Registros Actualizados: (840817, 62)


In [ ]:
# dt_total_Septiembre.to_csv(path+'seguimiento_corte_Septiembre.csv')

In [ ]:
dt_total_Septiembre = pd.read_csv(path+'seguimiento_corte_Septiembre.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,46,52,53,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
dt_total_Septiembre.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
dt_total_Septiembre.head(3)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,FECHA_INICIO,FUENTE,Emergia_Agosto_Bajas,Emergia_Agosto_Venta,Emergia_Agosto_Gestion,Emergia_Agosto_NoContacto,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,NaN,NaN,0,0,1,0,0,NaN,NaN,0
1,CC,1000719025,NATALIA MONTENEGRO LOPEZ,17659219,6.015875e+09,3.115875e+09,NaN,CIUDAD BOLIVAR,RAFAEL ESCAMILLA,CL 60A SUR 68 08 TO 3 AP 205,...,NaN,NaN,0,0,0,0,0,NaN,NaN,0
2,CC,79723833,JUAN CARLOS CASAS TORRES,13083282,6.013083e+09,3.152520e+09,NaN,TEUSAQUILLO,EL SALITRE,CL 41A 66A 49 AP 1005,...,NaN,NaN,0,0,0,0,0,NaN,NaN,0


In [ ]:
# dt_total_Septiembre[fechas]
# dt_total_Septiembre[fechas][(dt_total_Septiembre[fechas] == 0).sum(axis=1)>0].shape

In [ ]:
# fechas = ['Emergia_Mayo_Gestion', 'Emergia_Junio_Gestion', 'Emergia_Julio_Gestion', 'Emergia_Agosto_Gestion']
# fechas_= ['Emergia_Mayo_NoContacto', 'Emergia_Junio_NoContacto', 'Emergia_Julio_NoContacto', 'Emergia_Agosto_NoContacto'] 

In [ ]:
# (dt_total_Septiembre[fechas_][dt_total_Septiembre[fechas]==1].fillna(0).sum(axis=1)==0).sum()

In [ ]:
# dt_total_Septiembre[dt_total_Septiembre[]==1].shape

In [ ]:
df_t_ = dt_total_Septiembre.copy()

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
df_t_.Emergia_Septiembre_nuevo.sum()

19544

In [ ]:
datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Mayo_Bajas!=1]

<ipython-input-29-08b529afaa18>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Mayo_Bajas!=1]


In [ ]:
datos.shape

(609865, 61)

In [ ]:
conteo.shape

(609865, 2)

In [ ]:
df_t_['conteo'] = 0

In [ ]:
conteo.sort_values(by=['indices'], inplace=True)

In [ ]:
df_t_['conteo'].loc[datos.index] = conteo.conteo.tolist()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Mayo_Bajas!=1]

<ipython-input-40-08b529afaa18>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Mayo_Bajas!=1]


In [ ]:
datos.conteo.value_counts()

2.0    299563
3.0    190733
1.0    119492
4.0        77
Name: conteo, dtype: int64

In [ ]:
df_t_.shape

(840817, 62)

In [ ]:
dt_total_Septiembre.Emergia_Junio_Venta.sum()

3019

In [ ]:
df_t_.head(1)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,FECHA_INICIO,FUENTE,Emergia_Agosto_Bajas,Emergia_Agosto_Venta,Emergia_Agosto_Gestion,Emergia_Agosto_NoContacto,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,NaN,NaN,0,0,1,0,0,NaN,NaN,0


In [ ]:
ventas = ['Emergia_Mayo_Venta', 'Emergia_Junio_Venta', 'Emergia_Julio_Venta', 'Emergia_Agosto_Venta']
dt_total_Septiembre[ventas].sum().sum()

12542.0

In [ ]:
nocontacto = ['Emergia_Mayo_NoContacto', 'Emergia_Junio_NoContacto', 'Emergia_Julio_NoContacto', 'Emergia_Agosto_NoContacto']
(dt_total_Septiembre[nocontacto].sum(axis=1)>0).sum()

231087

In [ ]:
dt_total_Septiembre.Emergia_Septiembre_nuevo

0         0
1         0
2         0
3         0
4         0
         ..
615493    0
615494    0
615509    0
615514    0
615519    0
Name: Emergia_Septiembre_nuevo, Length: 840817, dtype: int64

In [ ]:
Gest = ['Emergia_Mayo_Gestion', 'Emergia_Junio_Gestion', 'Emergia_Julio_Gestion', 'Emergia_Agosto_Gestion', 'Emergia_Septiembre_nuevo']
dt_total_Septiembre[Gest].sum(axis=1).value_counts()

1.0    370204
0.0    277526
2.0    192204
3.0       883
dtype: int64

In [ ]:
Data = pd.DataFrame()
Data['data'] = dt_total_Septiembre[Gest].sum(axis=1)

In [ ]:
Data.groupby(by=['data'])['data'].count()

data
0.0    277526
1.0    370204
2.0    192204
3.0       883
Name: data, dtype: int64

In [ ]:
dt_total_Septiembre[['Emergia_Mayo_Gestion','Emergia_Junio_Gestion','Emergia_Julio_Gestion','Emergia_Agosto_Gestion']].sum()

Emergia_Mayo_Gestion      161540.0
Emergia_Junio_Gestion     202724.0
Emergia_Julio_Gestion     145474.0
Emergia_Agosto_Gestion    227979.0
dtype: float64

In [ ]:
datos['TELEFONO'] = datos['TELEFONO'].astype(str)
datos["TELEFONO"] = datos["TELEFONO"].apply(lambda x: x.replace(".0",""))

In [ ]:
datos.shape

(609865, 62)

In [ ]:
(datos['TELEFONO']=='-1').sum()

51

In [ ]:
datos = datos[datos['TELEFONO']!='-1']

In [ ]:
datos.columns

Index(['TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NOMBRE_CLIENTE', 'TELEFONO',
       'TELEFONO_CONTACTO_1', 'TELEFONO_CONTACTO_2', 'TELEFONO_CONTACTO_3',
       'LOCALIDAD', 'BARRIO', 'DIRECCION_INSTALACION', 'LATITUD', 'LONGITUD',
       'PAQUETE', 'PAQUETE_PLAN', 'CICLO', 'ESTADO', 'HABEAS_DATA',
       'MOVIL_ETB_ACTUAL', 'MOVIL_ETB_HISTORICO', 'Tipo_oferta',
       'Antiguedad_meses', 'VELOCIDAD', 'TECNOLOGIA', 'TARIFA_CONFIGURADA',
       'ESTRATO_USO', 'celular_1', 'celular_2', 'celular_3', 'C_SEG_UEN',
       'Localidad_zona', 'tipo_estrato', 'Tipo_velocidad', 'grupos', 'pqr',
       'TELEFONO2', 'Emergia_Mayo_nuevo', 'Emergia_Mayo_Bajas',
       'Emergia_Mayo_Venta', 'Emergia_Mayo_Gestion', 'Emergia_Mayo_NoContacto',
       'Emergia_Junio_nuevo', 'Emergia_Junio_Bajas', 'Emergia_Junio_Venta',
       'Emergia_Junio_Gestion', 'Emergia_Junio_NoContacto',
       'Emergia_Julio_nuevo', 'CORREO_ELECTRONICO', 'Emergia_Julio_Bajas',
       'Emergia_Julio_Venta', 'Emergia_Julio_Gestion',
  

In [ ]:
Gest = ['Emergia_Mayo_Gestion', 'Emergia_Junio_Gestion', 'Emergia_Julio_Gestion', 
        'Emergia_Agosto_Gestion', 'Mayorel_Gestionado_Mayo', 'Mayorel_Gestionado_Junio', 
        'Mayorel_Gestionado_Julio', 'Mayorel_Gestionado_Agosto']
datos[Gest].sum(axis=1)

In [ ]:
datos.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221025 609K.xlsx')

In [ ]:
datos_ = datos[['NUMERO_DOCUMENTO', 'TELEFONO', 'TELEFONO_CONTACTO_1', 'TELEFONO_CONTACTO_2', 'TELEFONO_CONTACTO_3',
                'celular_1', 'celular_2', 'celular_3']]

In [ ]:
datos_.to_excel(path+'Base Total de Clientes 609K_v2.xlsx')

In [ ]:
# gestiones a mes de septiembre
119492 1
299563 2
190733 3
77 4

In [ ]:
df_t_.shape

In [ ]:
df_t_.TELEFONO.unique()

In [ ]:
#@title ###Carga del retorno del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Retorno' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m_retorno = pd.read_excel(path+nombre_completo)
df_m_retorno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259932 entries, 0 to 259931
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ID BASE                   259932 non-null  object        
 1   NRO REGISTRO              259932 non-null  object        
 2   CAMPANA                   259932 non-null  object        
 3   PROVEEDOR                 259932 non-null  object        
 4   NOMBRE CLIENTE            259520 non-null  object        
 5   TIPO DOCUMENTO CLIENTE    259932 non-null  object        
 6   NUMERO DOCUMENTO CLIENTE  259932 non-null  object        
 7   ORI CORREO ELECTRONICO    220281 non-null  object        
 8   SEGMENTO                  239516 non-null  object        
 9   CANTIDAD LINEAS           259884 non-null  object        
 10  CELULAR                   259932 non-null  int64         
 11  TELEFONO1                 244519 non-null  float64       
 12  TE

In [ ]:
df_t_.columns

Index(['TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NOMBRE_CLIENTE', 'TELEFONO',
       'TELEFONO_CONTACTO_1', 'TELEFONO_CONTACTO_2', 'TELEFONO_CONTACTO_3',
       'LOCALIDAD', 'BARRIO', 'DIRECCION_INSTALACION', 'LATITUD', 'LONGITUD',
       'PAQUETE', 'PAQUETE_PLAN', 'CICLO', 'ESTADO', 'HABEAS_DATA',
       'MOVIL_ETB_ACTUAL', 'MOVIL_ETB_HISTORICO', 'Tipo_oferta',
       'Antiguedad_meses', 'VELOCIDAD', 'TECNOLOGIA', 'TARIFA_CONFIGURADA',
       'ESTRATO_USO', 'celular_1', 'celular_2', 'celular_3', 'C_SEG_UEN',
       'Localidad_zona', 'tipo_estrato', 'Tipo_velocidad', 'grupos', 'pqr',
       'TELEFONO2', 'Emergia_Mayo_nuevo', 'Emergia_Mayo_Bajas',
       'Emergia_Mayo_Venta', 'Emergia_Mayo_Gestion', 'Emergia_Mayo_NoContacto',
       'Emergia_Junio_nuevo', 'Emergia_Junio_Bajas', 'Emergia_Junio_Venta',
       'Emergia_Junio_Gestion', 'Emergia_Junio_NoContacto',
       'Emergia_Julio_nuevo', 'CORREO_ELECTRONICO', 'Emergia_Julio_Bajas',
       'Emergia_Julio_Venta', 'Emergia_Julio_Gestion',
  

In [ ]:
df_m_retorno.shape

In [ ]:
(df_m_retorno.CONTACTADO=='SI').sum()

In [ ]:
df_m_retorno.columns

In [ ]:
df_m_retorno.GESTIONADO.unique()

In [ ]:
df_m_retorno.drop_duplicates(subset=['NUMERO DOCUMENTO CLIENTE', 'NUMERO CONEXION']).shape

In [ ]:
(df_m_retorno.CONTACTADO=='SI').sum()

In [ ]:
df_t_.shape

(840817, 62)

In [ ]:
df_t_.columns

Index(['TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NOMBRE_CLIENTE', 'TELEFONO',
       'TELEFONO_CONTACTO_1', 'TELEFONO_CONTACTO_2', 'TELEFONO_CONTACTO_3',
       'LOCALIDAD', 'BARRIO', 'DIRECCION_INSTALACION', 'LATITUD', 'LONGITUD',
       'PAQUETE', 'PAQUETE_PLAN', 'CICLO', 'ESTADO', 'HABEAS_DATA',
       'MOVIL_ETB_ACTUAL', 'MOVIL_ETB_HISTORICO', 'Tipo_oferta',
       'Antiguedad_meses', 'VELOCIDAD', 'TECNOLOGIA', 'TARIFA_CONFIGURADA',
       'ESTRATO_USO', 'celular_1', 'celular_2', 'celular_3', 'C_SEG_UEN',
       'Localidad_zona', 'tipo_estrato', 'Tipo_velocidad', 'grupos', 'pqr',
       'TELEFONO2', 'Emergia_Mayo_nuevo', 'Emergia_Mayo_Bajas',
       'Emergia_Mayo_Venta', 'Emergia_Mayo_Gestion', 'Emergia_Mayo_NoContacto',
       'Emergia_Junio_nuevo', 'Emergia_Junio_Bajas', 'Emergia_Junio_Venta',
       'Emergia_Junio_Gestion', 'Emergia_Junio_NoContacto',
       'Emergia_Julio_nuevo', 'CORREO_ELECTRONICO', 'Emergia_Julio_Bajas',
       'Emergia_Julio_Venta', 'Emergia_Julio_Gestion',
  

# Octubre

In [ ]:
Mes = 'Octubre' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['ETB_Total_187k.xlsx',
 'ETB_Total_No_concactados_octubre_162k.xlsx',
 'ETB_Total_No_concactados_octubre_162k_2.xlsx',
 'ETB_Total_No_concactados_octubre_166k_3.xlsx',
 'ETB_Total_No_concactados_octubre_81k_S1.xlsx',
 'ETB_Total_No_concactados_octubre_81k_S2.xlsx',
 'Base Total de Clientes 609K.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221028.xlsx',
 'ETB_Total_seguimiento_octubre_final_599k.csv',
 'ETB_Total_seguimiento_octubre_final_661k.csv',
 'Retorno Octubre.xlsx',
 '609k_mejor gestion.xlsx',
 'R_Octubre2.xlsx',
 'Base Total de Clientes 609K_v2.xlsx']

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = "Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221028.xlsx" #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

In [ ]:
conteo = pd.read_csv(path+'Seguimiento_conteo_.csv')

In [ ]:
conteo.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
conteo.shape

(609865, 2)

In [ ]:
conteo.head(1)

,conteo,indices
0,1.0,233060


In [ ]:
datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1]

<ipython-input-79-d433eb06bbbb>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1]


In [ ]:
datos.reset_index(drop=True, inplace=True)

In [ ]:
datos.shape

(609865, 62)

In [ ]:
conteo.sort_values(by=['indices'], inplace=True)

In [ ]:
conteo.conteo.value_counts().sort_index()

1.0    119492
2.0    299563
3.0    190733
4.0        77
Name: conteo, dtype: int64

In [ ]:
conteo

,conteo,indices
466122,3.0,0
126246,2.0,1
126247,2.0,2
514025,3.0,3
330342,2.0,4
...,...,...
330336,2.0,609860
330341,2.0,609861
330343,2.0,609862
169547,2.0,609863


In [ ]:
datos = temporal[temporal.TELEFONO.notna()][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_nuevo!=1]

<ipython-input-75-a873c3aaad35>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos = temporal[temporal.TELEFONO.notna()][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_nuevo!=1]


In [ ]:
temporal.shape

(892644, 53)

In [ ]:
datos.head(2)

,Unnamed: 0,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,...,Emergia_Agosto_nuevo,FECHA_INICIO,FUENTE,Emergia_Agosto_Bajas,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas,Emergia_Octubre_nuevo,Emergia_Octubre_Bajas
0,0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,...,0,NaN,NaN,0,0,NaN,NaN,0,0,0
1,1,CC,1000719025,NATALIA MONTENEGRO LOPEZ,17659219,6.015875e+09,3.115875e+09,NaN,CIUDAD BOLIVAR,RAFAEL ESCAMILLA,...,0,NaN,NaN,0,0,NaN,NaN,0,0,0


In [ ]:
datos = datos.merge(conteo, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')

In [ ]:
datos.conteo.value_counts()

2.0    263643
1.0    119490
Name: conteo, dtype: int64

In [ ]:
datos['conteo'] = conteo[['conteo']]

In [ ]:
conteo.conteo.value_counts()

2.0    299563
3.0    190733
1.0    119492
4.0        77
Name: conteo, dtype: int64

In [ ]:
datos.sort_values(by=['conteo'], inplace=True)

In [ ]:
datos.shape

(609865, 51)

In [ ]:
df_m.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m.reset_index(drop=True, inplace=True)
df_m.shape

(633043, 36)

In [ ]:
df_t_.shape

(840817, 63)

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
df_t_.head(1)

,Unnamed: 0,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,...,FUENTE,Emergia_Agosto_Bajas,Emergia_Agosto_Venta,Emergia_Agosto_Gestion,Emergia_Agosto_NoContacto,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas,conteo
0,0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,...,NaN,0,0,1,0,0,NaN,NaN,0,3.0


In [ ]:
df_t_.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df_t_.TELEFONO = df_t_.TELEFONO.astype(str)
df_t_.NUMERO_DOCUMENTO = df_t_.NUMERO_DOCUMENTO.astype(str)
df_t_["TELEFONO"] = df_t_["TELEFONO"].apply(lambda x: x.replace(".0",""))
df_t_.reset_index(drop=True, inplace=True)
df_t_.shape

(840817, 62)

In [ ]:
# data con mes Septiembre
dft_st = df_t_[df_t_.TELEFONO.isna()]
dft_ct = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc),len(ind_con),len(indx))
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Registros nuevos: (17520, 36)


In [ ]:
dfm_st[[df_t_.columns[dfm_ct.columns.shape[0]:].values, 'Emergia_Octubre_nuevo']] = 0
df_New[[df_t_.columns[df_New.columns.shape[0]:].values, 'Emergia_Octubre_nuevo']] = 0
df_New['Emergia_Octubre_nuevo'] = 1
df_t_['Emergia_Octubre_nuevo'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [ ]:
temporal = pd.DataFrame()
temporal = pd.concat([df_t_, df_New, dfm_st], axis=0)
temporal.reset_index(drop=True, inplace=True)
temporal.shape

(892643, 63)

In [ ]:
dft_st = pd.concat([temporal[temporal.TELEFONO.isna()],temporal[temporal.TELEFONO=='nan'],
                    temporal[temporal.Emergia_Mayo_Bajas==1],
                    temporal[temporal.Emergia_Junio_Bajas==1], 
                    temporal[temporal.Emergia_Julio_Bajas==1], 
                    temporal[temporal.Emergia_Agosto_Bajas==1],
                    temporal[temporal.Emergia_Septiembre_Bajas==1]], axis=0)
dft_ct = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# datos = dft_ct[['TELEFONO','NUMERO_DOCUMENTO']].iloc[indx]
# y1 = datos['NUMERO_DOCUMENTO'].tolist()
# y2 = datos['TELEFONO'].tolist()
# indx = df_t.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
df_Eli = dft_ct.drop(indx,axis=0)
print('Registros eliminados: {}'.format(df_Eli.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


Registros eliminados: (28285, 63)


In [ ]:
dft_ct['Emergia_Octubre_Bajas'] = 0
dft_ct['Emergia_Octubre_Bajas'].iloc[df_Eli.index] = 1
dft_st['Emergia_Octubre_Bajas'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
temporal = pd.concat([dft_ct, dft_st], axis=0)
temporal.shape

(892643, 64)

In [ ]:
temporal.reset_index(drop=True, inplace=True)

In [ ]:
temporal[temporal.Emergia_Octubre_Bajas!=1][temporal.TELEFONO.notna()].conteo.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


2.0    284074
0.0    248472
3.0    183346
1.0    114084
4.0        76
Name: conteo, dtype: int64

In [ ]:
temporal[temporal.TELEFONO.notna()].shape

(858337, 63)

### ventas hasta octubre

In [ ]:
datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Octubre_nuevo!=1]
datos.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(581580, 64)

In [ ]:
ventas = pd.read_excel(path+'R_Octubre2.xlsx')

In [ ]:
ventas_r = ['MOVIL,ALTA POSPAGO', 'COMERCIAL,VENTA CRUZADA MOVIL']

In [ ]:
ventas_ = ventas.query("VENTA == @ventas_r")

In [ ]:
ventas_.rename(columns={'NUMERO DOCUMENTO CLIENTE':'NUMERO_DOCUMENTO'}, inplace=True)
ventas_.rename(columns={'NUMERO CONEXION':'TELEFONO'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
ventas_["TELEFONO"] = ventas_["TELEFONO"].astype(str)
ventas_["TELEFONO"] = ventas_["TELEFONO"].apply(lambda x: x.replace(".0",""))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
ventas_["NUMERO_DOCUMENTO"] = ventas_["NUMERO_DOCUMENTO"].astype(str)
ventas_["NUMERO_DOCUMENTO"] = ventas_["NUMERO_DOCUMENTO"].apply(lambda x: x.replace(".0",""))
ventas_.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
datos.drop(columns=['level_0',	'index'], axis=1, inplace=True)

In [ ]:
datos.head(1)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,Emergia_Agosto_Venta,Emergia_Agosto_Gestion,Emergia_Agosto_NoContacto,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas,conteo,Emergia_Octubre_nuevo,Emergia_Octubre_Bajas
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,0,1,0,0,NaN,NaN,0,3.0,0,0


In [ ]:
ventas_[ventas_.TELEFONO!='nan']

,NUMERO_DOCUMENTO,TELEFONO,VENTA,Unnamed: 3
387,73584665,17389391,"MOVIL,ALTA POSPAGO",MIX FTTH_NW
388,80256156,13470289,"MOVIL,ALTA POSPAGO",MIX FTTH_NW
389,1000223909,17315070,"MOVIL,ALTA POSPAGO",MIX FTTH_NW
390,1000718787,17642188,"MOVIL,ALTA POSPAGO",MIX FTTH_NW
391,1032449925,17927523,"MOVIL,ALTA POSPAGO",MIX FTTH_NW
...,...,...,...,...
2798,79811625,16791514,"MOVIL,ALTA POSPAGO",10_22_Base_MIX_SHB_C3
2799,51974853,14426696,"MOVIL,ALTA POSPAGO",10_22_Base_MIX_SHB_C3
2800,1023869066,13224752,"MOVIL,ALTA POSPAGO",10_22_Base_MIX_SHB_C3
2801,21491962,17814159,"MOVIL,ALTA POSPAGO",10_22_Base_MIX_SHB_C3


In [ ]:
temporal[(temporal.TELEFONO.notna()) | (temporal.TELEFONO!='nan')].shape

(892643, 64)

In [ ]:
dft_st = temporal[(temporal.TELEFONO.isna()) | (temporal.TELEFONO=='nan')]
dft_ct = temporal[(temporal.TELEFONO.notna()) | (temporal.TELEFONO!='nan')][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Octubre_nuevo!=1]
dfm_st = ventas_[ventas_.TELEFONO.isna()]
dfm_ct = ventas_[ventas_.TELEFONO.notna()]
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# datos = dft_ct[['TELEFONO','NUMERO_DOCUMENTO']].iloc[indx]
# y1 = datos['NUMERO_DOCUMENTO'].tolist()
# y2 = datos['TELEFONO'].tolist()
# indx = df_t.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
# print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
# df_Eli = dft_ct.drop(indx,axis=0)
# print('Registros nuevos: {}'.format(df_New.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
temporal['ventas_a_octubre'] = 0
temporal['ventas_a_octubre'].loc[indx] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
temporal[temporal['ventas_a_octubre']!=1].conteo.value_counts()

2.0    274926
0.0    194263
3.0    190005
1.0    118072
4.0        77
Name: conteo, dtype: int64

In [ ]:
temporal.head(1)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,Emergia_Agosto_Gestion,Emergia_Agosto_NoContacto,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas,conteo,Emergia_Octubre_nuevo,Emergia_Octubre_Bajas,ventas_a_octubre
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,1,0,0,NaN,NaN,0,3.0,0,0,0


In [ ]:
df_t_ = temporal.copy()

In [ ]:
df_New.shape

(828, 4)

In [ ]:
df_New.shape

(581582, 52)

In [ ]:
df_New.drop(columns=['index'], axis=1, inplace=True)

In [ ]:
df_New.iloc[:,:34].head(1)

,NUMERO_DOCUMENTO,NUMERO CONEXION,VENTA,Unnamed: 3
0,1020716780,NaN,"MOVIL,ALTA POSPAGO",MOVIL_WINBACK_OCTUBRE


In [ ]:
df_New.conteo.value_counts()

In [ ]:
# temporal.to_csv(path+'ETB_Total_cruce_octubre.csv')

In [ ]:
# temporal=pd.read_csv(path+'ETB_Total_cruce_octubre.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,35,41,44,45,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
temporal[temporal.Emergia_Octubre_Bajas!=1][temporal.TELEFONO.notna()].conteo.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


2.0    286805
3.0    180151
1.0    114553
0.0     17521
4.0        73
Name: conteo, dtype: int64

In [ ]:
temporal[temporal.TELEFONO.]

In [ ]:
datos = pd.read_excel(path+'Retorno_Octubre.xlsx', sheet_name=1)

In [ ]:
datos.shape

(316179, 3)

In [ ]:
datos = datos[datos.VENTA==1]
datos['NUMERO CONEXION'] = datos['NUMERO CONEXION'].astype(str)
datos["NUMERO CONEXION"] = datos["NUMERO CONEXION"].apply(lambda x: x.replace(".0",""))

In [ ]:
datos.rename(columns={'NUMERO CONEXION':'TELEFONO'}, inplace=True)

In [ ]:
datos.rename(columns={'NUMERO DOCUMENTO CLIENTE':'NUMERO_DOCUMENTO'}, inplace=True)

In [ ]:
import gc 

In [ ]:
gc.collect()

1018

In [ ]:
datos_org = temporal[temporal.Emergia_Octubre_Bajas!=1][temporal.TELEFONO.notna()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
datos.columns

Index(['NUMERO_DOCUMENTO', 'TELEFONO', 'VENTA'], dtype='object')

In [ ]:
datos['NUMERO_DOCUMENTO'] = datos['NUMERO_DOCUMENTO'].astype(str)

In [ ]:
datos_org.shape

(599103, 53)

In [ ]:
path

'Data/ETB_Total/2022/Noviembre/'

In [ ]:
temporal.shape

(661692, 53)

In [ ]:
# datos_org.to_csv(path+'ETB_Total_seguimiento_octubre_final_599k.csv')

In [ ]:
temporal.to_csv(path+'ETB_Total_seguimiento_octubre_final_661k.csv')

In [ ]:
# data con mes Septiembre
dft_st = datos_org[datos_org.TELEFONO.isna()]
dft_ct = datos_org[datos_org.TELEFONO.notna()]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = datos[datos.TELEFONO.isna()]
dfm_ct = datos[datos.TELEFONO.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc),len(ind_con),len(indx))
df_New = dft_ct.drop(indx, axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

42915 3322 3104
Registros nuevos: (595999, 53)


In [ ]:
df_New.conteo.value_counts()

2.0    285672
3.0    179009
1.0    113791
0.0     17459
4.0        68
Name: conteo, dtype: int64

In [ ]:
df_New.conteo.fillna(0, inplace=True)

In [ ]:
df_New[df_New.conteo!=0].sort_values(by=['conteo'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
base_nov = df_New.iloc[:138121,:34]

In [ ]:
base_nov['conteo'] = df_New['conteo'][:138121]

In [ ]:
base_nov.shape

(138121, 35)

In [ ]:
base_nov = check(base_nov)

In [ ]:
base_nov = main(base_nov)

cantidad de registros: 138121 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 137946 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 102369 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 112866 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 68732 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 102369 

cantidad de registros telefono 2: 112866 

cantidad de registros telefono 3: 68732 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 102369 


In [ ]:
path

'Data/ETB_Total/2022/Noviembre/'

In [ ]:
base_nov.to_csv(path+'Base_ETB_Total_Noviembre_138k_normalizada.csv')

In [ ]:
base_nov.to_excel(path+'Base_ETB_Total_Noviembre_138k_normalizada.xlsx')

In [ ]:
temporal[temporal.TELEFONO.notna()][temporal.Emergia_Octubre_Bajas!=1].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(599103, 53)

In [ ]:
temporal.conteo.fillna(0)
temporal.conteo.value_counts()

2.0    290065
3.0    181586
1.0    115761
0.0     51827
4.0        69
Name: conteo, dtype: int64

In [ ]:
temporal.head(2)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,FECHA_INICIO,FUENTE,Emergia_Agosto_Bajas,Emergia_Agosto_Venta,Emergia_Agosto_Gestion,Emergia_Agosto_NoContacto,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,NaT,NaN,0,0,1,0,0,NaN,NaN,0
1,CC,1000719025,NATALIA MONTENEGRO LOPEZ,17659219,6.015875e+09,3.115875e+09,NaN,CIUDAD BOLIVAR,RAFAEL ESCAMILLA,CL 60A SUR 68 08 TO 3 AP 205,...,NaT,NaN,0,0,0,0,0,NaN,NaN,0


In [ ]:
path

'Data/ETB_Total/2022/Octubre/'

In [ ]:
dt_total_Octubre = temporal.copy()
print('Registros Actualizados: {}'.format(dt_total_Octubre.shape))

Registros Actualizados: (892644, 52)


In [ ]:
dt_total_Septiembre.to_csv(path+'seguimiento_corte_Septiembre.csv')

In [ ]:
dt_total_Septiembre = pd.read_csv(path+'seguimiento_corte_Septiembre.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,46,52,53,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
conteo = pd.read_csv(path+'Conteo_septiembre.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
conteo.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
conteo.rename(columns={'NUMERO_DOCUMENT':'NUMERO_DOCUMENTO'}, inplace=True)

In [ ]:
conteo

,NUMERO_DOCUMENTO,TELEFONO,conteo
0,1031168138,12020249,1.0
1,1022380530,14494391,1.0
2,45755487,86703477,1.0
3,17117555,12076412,1.0
4,1015475091,16297730,1.0
...,...,...,...
610362,19468189,13462791,4.0
610363,19236156,14333172,4.0
610364,80410278,16371258,4.0
610365,52315512,14575353,4.0


In [ ]:
#@title ###Carga del retorno del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Retorno' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m_retorno = pd.read_excel(path+nombre_completo)
df_m_retorno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193831 entries, 0 to 193830
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ID BASE                   193831 non-null  object        
 1   NRO REGISTRO              193831 non-null  object        
 2   CAMPANA                   193831 non-null  object        
 3   PROVEEDOR                 193831 non-null  object        
 4   NOMBRE CLIENTE            193524 non-null  object        
 5   TIPO DOCUMENTO CLIENTE    193815 non-null  object        
 6   NUMERO DOCUMENTO CLIENTE  193815 non-null  object        
 7   ORI CORREO ELECTRONICO    178942 non-null  object        
 8   SEGMENTO                  181888 non-null  object        
 9   CANTIDAD LINEAS           188212 non-null  object        
 10  CELULAR                   193831 non-null  int64         
 11  TELEFONO1                 184496 non-null  float64       
 12  TE

In [ ]:
df_m_retorno.shape

(193831, 57)

In [ ]:
(df_m_retorno.CONTACTADO=='SI').sum()

89539

In [ ]:
df_m_retorno.columns

Index(['ID BASE', 'NRO REGISTRO', 'CAMPANA', 'PROVEEDOR', 'NOMBRE CLIENTE',
       'TIPO DOCUMENTO CLIENTE', 'NUMERO DOCUMENTO CLIENTE',
       'ORI CORREO ELECTRONICO', 'SEGMENTO', 'CANTIDAD LINEAS', 'CELULAR',
       'TELEFONO1', 'TELEFONO2', 'TELEFONO3', 'DEPARTAMENTO',
       'CIUDAD INSTALACION', 'CICLO', 'NOMBRE PLAN ACTUAL', 'PROMEDIO FACT',
       'PLAN SUGERIDO1', 'VLR OFERTA1', 'PLAN SUGERIDO2', 'VLR OFERTA',
       'PLAN SUGERIDO3', 'VLR OFERTA3', 'ID BASE2', 'NRO REGISTRO3',
       'CAMPANACANAL', 'TELEFONOCONTRATADO', 'PROVEEDORCANAL', 'GESTIONADO',
       'FECHA HORA GESTION', 'CONTACTADO', 'TIPO GESTION', 'OFERTA CONTRATADA',
       'AGRUPACION', 'CAMPO', 'ASESOR', 'CODIGO GESTION', 'NRO PEDIDO',
       'NRO CUN', 'INTENTOS', 'COD PRODUCTO', 'PRODUCTO', 'INTENTOS4', 'NODO',
       'NUMERO CONEXION', 'USER ID', 'ID LLAMADA', 'duracion llamada', 'fcel',
       'ftel1', 'ftel2', 'ftel3', 'fconex', 'LOCALIDAD', 'GRUPO'],
      dtype='object')

In [ ]:
df_m_retorno.GESTIONADO.unique()

array(['SI', 'NO'], dtype=object)

In [ ]:
df_m_retorno.drop_duplicates(subset=['NUMERO DOCUMENTO CLIENTE', 'NUMERO CONEXION']).shape

(175258, 57)

In [ ]:
(df_m_retorno.CONTACTADO=='SI').sum()

89539

In [ ]:
datos.shape

(609865, 62)

In [ ]:
datos.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
temporal = pd.read_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221031_seguimiento_892k.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,35,41,44,45,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
temporal.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
temporal.head(2)

,Unnamed: 0,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,...,Emergia_Agosto_nuevo,FECHA_INICIO,FUENTE,Emergia_Agosto_Bajas,Emergia_Septiembre_nuevo,id_sep,key_sep,Emergia_Septiembre_Bajas,Emergia_Octubre_nuevo,Emergia_Octubre_Bajas
0,0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,...,0,NaN,NaN,0,0,NaN,NaN,0,0,0
1,1,CC,1000719025,NATALIA MONTENEGRO LOPEZ,17659219,6.015875e+09,3.115875e+09,NaN,CIUDAD BOLIVAR,RAFAEL ESCAMILLA,...,0,NaN,NaN,0,0,NaN,NaN,0,0,0


In [ ]:
mejor_ges = pd.read_excel(path+'Base Total de Clientes 609K_v2.xlsx')

In [ ]:
mejor_ges.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
mejor_ges.columns

Index(['NUMERO_DOCUMENTO', 'TELEFONO', 'TELEFONO_CONTACTO_1',
       'TELEFONO_CONTACTO_2', 'TELEFONO_CONTACTO_3', 'celular_1', 'celular_2',
       'celular_3', '||', 'qcode', 'n1', 'n2', 'n3'],
      dtype='object')

In [ ]:
mejor_ges["NUMERO_DOCUMENTO"] = mejor_ges["NUMERO_DOCUMENTO"].astype(str)

In [ ]:
datos["NUMERO_DOCUMENTO"] = datos["NUMERO_DOCUMENTO"].astype(str)

In [ ]:
datos["TELEFONO"] = datos["TELEFONO"].apply(lambda x: x.replace(".0",""))

In [ ]:
datos.drop(labels=['index'], axis=1, inplace=True)

In [ ]:
df_New = check(df_New)

In [ ]:
df_New = main(df_New)

In [ ]:
df_New.rename(columns={'NUMERO_CONEXION':'TELEFONO'}, inplace=True)

In [ ]:
df_New.reset_index(inplace=True)
mejor_ges.reset_index(inplace=True)

In [ ]:
# Cruce mejor gestion octubre 609K
data_cruce = df_New.iloc[:,:35].merge(mejor_ges[['NUMERO_DOCUMENTO', 'TELEFONO', 'qcode', 'n1', 'n2', 'n3']],  right_on=['NUMERO_DOCUMENTO','TELEFONO'], left_on=['NUMERO_DOCUMENTO','TELEFONO'], how='left')

In [ ]:
data_cruce.drop(labels=['level_0', 'index'], axis=1, inplace=True)

In [ ]:
data_cruce.shape

(581582, 37)

In [ ]:
mejor_ges[mejor_ges.n1.notna()].shape

(341638, 14)

In [ ]:
data_cruce.head(3)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,celular_3,C_SEG_UEN,Localidad_zona,tipo_estrato,Tipo_velocidad,grupos,qcode,n1,n2,n3
0,CC,51776541,FLOR ESTHER CASTRO,13561955,6.013562e+09,3.214638e+09,3.107905e+09,KENNEDY,GALAN,KR 97F 34A 40 SUR CA J27,...,NaN,HOGARES,zona centro sur,medio,Menor 100,Otros,2024.0,NO VENTA,COMERCIAL,NO ACEPTA CONDICIONES DE LA OFERTA
1,CC,1000719025,NATALIA MONTENEGRO LOPEZ,17659219,6.015875e+09,3.115875e+09,NaN,CIUDAD BOLIVAR,RAFAEL ESCAMILLA,CL 60A SUR 68 08 TO 3 AP 205,...,NaN,HOGARES,zona centro sur,medio,Mayor 100,FTTH - Dúo - zona centro sur - medio - Mayor 100,2018.0,NO VENTA,CLIENTE,NO VOLVER A LLAMAR
2,CC,79723833,JUAN CARLOS CASAS TORRES,13083282,6.013083e+09,3.152520e+09,NaN,TEUSAQUILLO,EL SALITRE,CL 41A 66A 49 AP 1005,...,NaN,HOGARES,zona norte,alto,Mayor 100,FTTH - Dúo - zona norte - alto - Mayor 100,2025.0,NO VENTA,COMERCIAL,"NO ACEPTA, NO INDICA RAZON"


In [ ]:
data_cruce[data_cruce.qcode.notna()].shape

(581531, 37)

In [ ]:
data_cruce = data_cruce[data_cruce.qcode.notna()]
data_cruce.reset_index(inplace=True)

In [ ]:
data_cruce.shape

(581531, 38)

In [ ]:
data_cruce.to_csv(path+'ETB Total Cruce actual con tipificacion noviembre.csv')

In [ ]:
data_cruce.to_excel(path+'ETB Total Cruce actual con tipificacion noviembre.xlsx')

In [ ]:
path

'Data/ETB_Total/2022/Noviembre/'

In [ ]:
data_cruce.shape

(78608, 39)

# Noviembre

In [ ]:
Mes = 'Noviembre' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['Base Total de Clientes de la OperaciÃ³n Fija_20221031_new_17k.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221031_new_17k_normalizada.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221031_seguimiento_892k.csv',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221031_actualizada_599k.csv',
 'ETB_Total_cruce_octubre.csv',
 'Base_ETB_Total_Noviembre_138k_normalizada.csv',
 'Base_ETB_Total_Noviembre_138k_normalizada.xlsx',
 'Base Total de Cliente con cruce de ventas a octubre 2022.xlsx',
 'ETB Total Cruce actual con tipificacion noviembre.csv',
 'ETB Total Cruce actual con tipificacion noviembre.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130.xlsx']

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = "Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221130" #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633480 entries, 0 to 633479
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   TIPO_DOCUMENTO         633480 non-null  object        
 1   NUMERO_DOCUMENTO       633480 non-null  object        
 2   NOMBRE_CLIENTE         632693 non-null  object        
 3   TELEFONO               594987 non-null  object        
 4   TELEFONO_CONTACTO_1    538419 non-null  float64       
 5   TELEFONO_CONTACTO_2    550971 non-null  float64       
 6   TELEFONO_CONTACTO_3    406425 non-null  float64       
 7   LOCALIDAD              576223 non-null  object        
 8   BARRIO                 557736 non-null  object        
 9   DIRECCION_INSTALACION  633387 non-null  object        
 10  LATITUD                573413 non-null  float64       
 11  LONGITUD               573413 non-null  float64       
 12  PAQUETE                633480 non-null  obje

In [ ]:
temporal = pd.read_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221031_actualizada_599k.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
ventas = pd.read_excel(path+'INFO VENTAS 2022.xlsx')

In [ ]:
ventas = ventas[['MESVENTA','cedula']]

In [ ]:
ventas.rename(columns ={'cedula':'NUMERO_DOCUMENTO'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
datos = temporal.merge(ventas, right_on=['NUMERO_DOCUMENTO'], left_on=['NUMERO_DOCUMENTO'], how='left')

In [ ]:
datos = check(datos)

In [ ]:
datos = main(datos)

cantidad de registros: 599779 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 597510 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 427237 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 502869 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 301467 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 427237 

cantidad de registros telefono 2: 502869 

cantidad de registros telefono 3: 301467 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 427237

In [ ]:
datos[datos.MESVENTA.notna()].to_excel(path+'Base Total de Cliente con cruce de ventas a octubre 2022.xlsx')

In [ ]:
datos[datos.MESVENTA.notna()][datos.ESTRATO_USO==3].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(3701, 36)

In [ ]:
datos.ESTRATO_USO[0]

2

In [ ]:
datos = datos.iloc[:,:35].merge(ventas, right_on=['NUMERO_DOCUMENTO',''], left_on=['NUMERO_DOCUMENTO'], how='left')

In [ ]:
temporal.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221031_actualizada.xlsx')

In [ ]:
temporal.shape

In [ ]:
# temporal.to_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221031_seguimiento_892k.csv')

In [ ]:
df_t_ = pd.read_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221031_seguimiento_892k.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,35,41,44,45,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Octubre_nuevo!=1].shape

<ipython-input-7-b119f184fefa>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Octubre_nuevo!=1].shape


(725470, 53)

In [ ]:
temporal[temporal.TELEFONO.notna()][temporal.Emergia_Octubre_nuevo!=1].count()

In [ ]:
df_New.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221031_new_17k.xlsx')

In [ ]:
df_New_ = check(df_New)

In [ ]:
df_New_ = main(df_New_)

cantidad de registros: 17521 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 16849 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 10777 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 16416 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 10136 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 10777 

cantidad de registros telefono 2: 16416 

cantidad de registros telefono 3: 10136 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 10777 

cantid

In [ ]:
df_New_.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221031_new_17k_normalizada.xlsx')

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1]

<ipython-input-10-7b5b6d8b888d>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1]


In [ ]:
datos.iloc[:,:34].shape

(599103, 34)

In [ ]:
datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_nuevo!=1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
datos.iloc[:,:34].to_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221031_actualizada_599k.csv')

In [ ]:
datos.shape

(599103, 52)

In [ ]:
conteo.columns

Index(['NUMERO_DOCUMENTO', 'TELEFONO', 'conteo'], dtype='object')

In [ ]:
datos = datos.merge(conteo, right_on=['NUMERO_DOCUMENTO', 'TELEFONO'], left_on=['NUMERO_DOCUMENTO', 'TELEFONO'], how='left')

In [ ]:
datos.conteo.fillna(0,inplace=True)

In [ ]:
datos.conteo.value_counts()

0.0    234620
2.0    150258
1.0    113623
3.0    100962
Name: conteo, dtype: int64

In [ ]:
datos.sort_values(by=['Emergia_Octubre_nuevo'], ascending=False, inplace=True)

## Actualización Para diciembre

In [ ]:
df_t_.shape

(892644, 53)

In [ ]:
df_t_.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

### Estrategía de 5cc en la actuallización

In [ ]:
df_m['Antiguedad_meses'] = 0

In [ ]:
estrategia_5cc = df_m[df_m.MOVIL_PLAN_ETB_ACTUAL.notna()]
estrategia_5cc = estrategia_5cc[df_t_.columns[:34].tolist()]

In [ ]:
estrategia_5cc.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221130_55k_Estrategia_5cc.xlsx')

In [ ]:
estrategia_5cc = check(estrategia_5cc)
estrategia_5cc = main(estrategia_5cc)

cantidad de registros: 55025 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 51414 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 46392 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 52978 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 44094 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 46392 

cantidad de registros telefono 2: 52978 

cantidad de registros telefono 3: 44094 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 46392 

cantid

In [ ]:
estrategia_5cc.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221130_55k_Estrategia_5cc_normalizada.xlsx')

In [ ]:
df_m['Antiguedad_meses'] = 0

In [ ]:
df_m = df_m[df_m.MOVIL_PLAN_ETB_ACTUAL.isna()]
df_m = df_m[df_t_.columns[:34].tolist()]

In [ ]:
df_m.columns

Index(['TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NOMBRE_CLIENTE', 'TELEFONO',
       'TELEFONO_CONTACTO_1', 'TELEFONO_CONTACTO_2', 'TELEFONO_CONTACTO_3',
       'LOCALIDAD', 'BARRIO', 'DIRECCION_INSTALACION', 'LATITUD', 'LONGITUD',
       'PAQUETE', 'PAQUETE_PLAN', 'CICLO', 'ESTADO', 'HABEAS_DATA',
       'MOVIL_ETB_ACTUAL', 'MOVIL_ETB_HISTORICO', 'Tipo_oferta',
       'Antiguedad_meses', 'VELOCIDAD', 'TECNOLOGIA', 'TARIFA_CONFIGURADA',
       'ESTRATO_USO', 'celular_1', 'celular_2', 'celular_3', 'C_SEG_UEN',
       'Localidad_zona', 'tipo_estrato', 'Tipo_velocidad', 'grupos', 'pqr'],
      dtype='object')

In [ ]:
df_m.shape

(578455, 34)

In [ ]:
df_m.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m.reset_index(drop=True, inplace=True)
df_m.shape

(578183, 34)

In [ ]:
df_t_.shape

(892643, 66)

In [ ]:
df_t_.sort_values(by=['conteo'], inplace=True)

In [ ]:
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
(df_t_.Emergia_Octubre_nuevo==1).sum()

17520

In [ ]:
df_t_['gestion_noviembre'][df_t_['gestion_noviembre'][df_t_.conteo>0][df_t_.conteo==1][:113791].index] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_t_['gestion_noviembre'] = 0

In [ ]:
df_t_['gestion_noviembre'][df_t_['gestion_noviembre'][df_t_.conteo>0][df_t_.conteo==1][:113791].index] = 1
df_t_['gestion_noviembre'][df_t_['gestion_noviembre'][df_t_.conteo>0][df_t_.conteo==2][:24330].index] = 1
df_t_['gestion_noviembre'][df_t_['gestion_noviembre'][df_t_.conteo>0][df_t_.Emergia_Octubre_nuevo==1].index] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
df_t_.gestion_noviembre.sum()

138121

In [ ]:
path

'Data/ETB_Total/2022/Noviembre/'

In [ ]:
df_t_.to_csv(path+'Seguimiento_ETB_Total_nov.csv')

In [ ]:
df_t_.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df_t_.TELEFONO = df_t_.TELEFONO.astype(str)
df_t_.NUMERO_DOCUMENTO = df_t_.NUMERO_DOCUMENTO.astype(str)
df_t_["TELEFONO"] = df_t_["TELEFONO"].apply(lambda x: x.replace(".0",""))
df_t_.reset_index(drop=True, inplace=True)
df_t_.shape

In [ ]:
datos = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1]
datos.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(748756, 54)

In [ ]:
# data con mes Septiembre
dft_st = df_t_[(df_t_.TELEFONO.isna()) | (df_t_.TELEFONO=='nan')]
dft_ct = df_t_[(df_t_.TELEFONO.notna()) | (df_t_.TELEFONO!='nan')][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc),len(ind_con),len(indx))
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


528983 527871 527108
Registros nuevos: (16075, 34)


In [ ]:
df_New.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221130_16k_nuevos.xlsx')

In [ ]:
df_New_ = df_New.copy()
df_New_ = check(df_New_)
df_New_ = main(df_New_)
df_New_.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221130_16k_nuevos_normalizada.xlsx')

In [ ]:
dfm_st[[df_t_.columns[dfm_ct.columns.shape[0]:].values, 'Emergia_Noviembre_nuevo']] = 0
df_New[[df_t_.columns[df_New.columns.shape[0]:].values, 'Emergia_Noviembre_nuevo']] = 0
df_New['Emergia_Noviembre_nuevo'] = 1
df_t_['Emergia_Noviembre_nuevo'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [ ]:
temporal = pd.DataFrame()
temporal = pd.concat([df_t_, df_New, dfm_st], axis=0)
temporal.reset_index(drop=True, inplace=True)
temporal.shape

(943718, 67)

In [ ]:
temporal.Emergia_Noviembre_nuevo.sum()

16075

In [ ]:
datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_nuevo!=1]
datos.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(599100, 67)

In [ ]:
temporal[temporal.TELEFONO=='nan'].shape

(149653, 54)

In [ ]:
dft_st.shape

(328541, 54)

In [ ]:
dft_st = pd.concat([temporal[temporal.TELEFONO=='nan'], temporal[temporal.TELEFONO.isna()],temporal[temporal.Emergia_Mayo_Bajas==1],
                    temporal[temporal.Emergia_Junio_Bajas==1], temporal[temporal.Emergia_Julio_Bajas==1], 
                    temporal[temporal.Emergia_Agosto_Bajas==1], temporal[temporal.Emergia_Septiembre_Bajas==1],
                    temporal[temporal.Emergia_Octubre_Bajas==1]], axis=0)
dft_ct = temporal[(temporal.TELEFONO.notna()) & (temporal.TELEFONO!='nan')][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# datos = dft_ct[['TELEFONO','NUMERO_DOCUMENTO']].iloc[indx]
# y1 = datos['NUMERO_DOCUMENTO'].tolist()
# y2 = datos['TELEFONO'].tolist()
# indx = df_t.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
df_Eli = dft_ct.drop(indx,axis=0)
print('Registros eliminados: {}'.format(df_Eli.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


549058 544386 543552
Registros eliminados: (71623, 67)


In [ ]:
dft_ct['Emergia_Noviembre_Bajas'] = 0
dft_ct['Emergia_Noviembre_Bajas'].iloc[df_Eli.index] = 1
dft_st['Emergia_Noviembre_Bajas'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
temporal = pd.concat([dft_ct, dft_st], axis=0)
temporal.shape

(943718, 68)

In [ ]:
temporal.reset_index(drop=True, inplace=True)

In [ ]:
datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Noviembre_nuevo!=1][temporal.Emergia_Octubre_nuevo!=1][temporal.gestion_noviembre!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
temporal['gestion_diciembre'] = 0  

In [ ]:
temporal.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
[temporal.conteo==2].iloc[:141143]

In [ ]:
temporal['gestion_diciembre'][temporal['gestion_diciembre'].loc[temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Noviembre_nuevo!=1][temporal.Emergia_Octubre_nuevo!=1][temporal.gestion_noviembre!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1][temporal.conteo==2].index][:141143].index] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
temporal['gestion_diciembre'].sum()

172143

In [ ]:
datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Noviembre_nuevo!=1][temporal.Emergia_Octubre_nuevo!=1][temporal.gestion_noviembre!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
datos.sort_values(by=['conteo'], ascending=False, inplace=True)

In [ ]:
temporal['gestion_diciembre'].loc[datos.iloc[:31000].index] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
datos.iloc[:31000,:34].to_excel(path+'mayorel_Diciembre_31k.xlsx')

In [ ]:
datos.iloc[:31000,:34].shape

(31000, 34)

In [ ]:
# almacenar para gestión de diciembre a 1 toque
datos[datos.conteo==2].iloc[:141143,:34].to_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221130_141k.csv')

In [ ]:
data_ = datos[datos.conteo==2].iloc[:141143,:34].copy()

In [ ]:
data_ = check(data_)
data_ = main(data_)
data_.to_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221130_141k_normalizada.csv')

cantidad de registros: 141143 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 140865 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 100277 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 119608 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 70518 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 100277 

cantidad de registros telefono 2: 119608 

cantidad de registros telefono 3: 70518 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 100277 


In [ ]:
temporal['gestion_diciembre'].sum()

172143

In [ ]:
temporal.to_csv(path+'Seguimiento_ETB_Total_dic.csv')

# Estratégia_ETB_tipificaciones_diciembre

In [ ]:
temporal = pd.read_csv(path+'Seguimiento_ETB_Total_dic.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,46,52,53,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Diciembre_Bajas!=1]
datos.shape

<ipython-input-89-1de37aa83bf6>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Diciembre_Bajas!=1]


(540938, 73)

In [ ]:
datos.columns

In [ ]:
temporal.Emergia_Noviembre_nuevo.sum()

16075

In [ ]:
temporal.Emergia_Octubre_nuevo.sum()

17520

In [ ]:
temporal.ventas_a_octubre.sum()

115300

In [ ]:
temporal['conteo'][temporal.ventas_a_octubre!=1][temporal.gestion_][temporal.gestion_diciembre!=1][temporal.Emergia_Noviembre_nuevo!=1][temporal.Emergia_Octubre_nuevo!=1].value_counts()

In [ ]:
datos['conteo_act_m'][datos.ventas_a_octubre!=1][datos.gestion_noviembre!=1][datos.gestion_diciembre!=1][datos.Emergia_Noviembre_nuevo!=1][datos.Emergia_Octubre_nuevo!=1][datos.Emergia_Diciembre_nuevo!=1].value_counts()

3.0    130405
2.0     79406
1.0      4634
Name: conteo_act_m, dtype: int64

In [ ]:
datos['conteo_act_m'][datos.ventas_a_octubre!=1][datos.gestion_noviembre!=1][datos.gestion_diciembre!=1][datos.Emergia_Noviembre_nuevo!=1][datos.Emergia_Diciembre_nuevo!=1].value_counts().sum()

215437

In [ ]:
datos_ = datos[datos.ventas_a_octubre!=1][datos.gestion_noviembre!=1][datos.gestion_diciembre!=1][datos.Emergia_Noviembre_nuevo!=1][datos.Emergia_Diciembre_nuevo!=1].iloc[:,:35]

<ipython-input-96-6c76d3ee8fb7>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos_ = datos[datos.ventas_a_octubre!=1][datos.gestion_noviembre!=1][datos.gestion_diciembre!=1][datos.Emergia_Noviembre_nuevo!=1][datos.Emergia_Diciembre_nuevo!=1].iloc[:,:35]


In [ ]:
datos_.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
datos_.reset_index(drop=True, inplace=True)

In [ ]:
datos_.head(2)

,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,DIRECCION_INSTALACION,...,celular_1,celular_2,celular_3,C_SEG_UEN,Localidad_zona,tipo_estrato,Tipo_velocidad,grupos,pqr,Emergia_Mayo_nuevo
0,CC,1072640226,NICOLAS AFANADOR REYES,16420795,6.013836e+09,3.108604e+09,3.108604e+09,CHAPINERO,LOS ROSALES,CL 79B 5 72 AP 502,...,3.108604e+09,NaN,NaN,HOGARES,zona norte,alto,Mayor 100,FTTH - Dúo - zona norte - alto - Mayor 100,0,NaN
1,CC,1013665196,CAMILO SNEYDER HERRERA MARTINEZ,17125410,6.017125e+09,3.209012e+09,3.209012e+09,USME,VILLA DIANA,KR 13 BIS ESTE 88G 71 SUR,...,NaN,NaN,NaN,HOGARES,zona centro sur,medio,Mayor 100,FTTC - Dúo,18,NaN


In [ ]:
datos_.shape

(215437, 35)

In [ ]:
datos_.to_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221214_220k_Estrategia_tipologias.csv')

In [ ]:
datos_ = check(datos_)
datos_ = main(datos_)

cantidad de registros: 220918 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 220742 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 163793 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 195785 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 110744 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 163793 

cantidad de registros telefono 2: 195785 

cantidad de registros telefono 3: 110744 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 163793

In [ ]:
datos_.to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221214_220k_Estrategia_tipologias_normalizada.xlsx')

In [ ]:
datos_[['TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NUMERO_CONEXION', 'telefono_1', 'telefono_2', 'telefono_3', 'celular_1', 'celular_2', 'celular_3']].to_excel(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221214_220k_Estrategia_tipologias_normalizada.xlsx')

In [ ]:
datos.gestion_noviembre.sum()

116018

In [ ]:
temporal.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
temporal.reset_index(drop=True, inplace=True)

In [ ]:
temporal['Emergia_Noviembre_Bajas'].sum()

71623

In [ ]:
temporal[temporal.TELEFONO.notna()][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_nuevo!=1].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(748756, 53)

In [ ]:
temporal.shape

(943719, 54)

In [ ]:
dt_total_Noviembre = temporal.copy()
print('Registros Actualizados: {}'.format(dt_total_Noviembre.shape))

Registros Actualizados: (943719, 54)


In [ ]:
datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
datos.iloc[:,:34].shape

(543572, 34)

In [ ]:
datos.iloc[:,:34].to_csv(path+'Base Total de Clientes de la OperaciÃ³n Fija_20221130_actualizada_543k.csv')

In [ ]:
datos.shape

(543572, 54)

# Diciembre

In [ ]:
Mes = 'Diciembre' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['Base Total de Clientes de la OperaciÃ³n Fija_20221130_55k_Estrategia_5cc.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130_55k_Estrategia_5cc_normalizada.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130_16k_nuevos.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130_16k_nuevos_normalizada.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130_47k_Estrategia_5cc_normalizada.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130_actualizada_543k.csv',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130_141k.csv',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130_141k_normalizada.csv',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221130_141k_normalizada.xlsx',
 'Seguimiento_ETB_Total_dic.csv',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221214_220k_Estrategia_tipologias.csv',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221214_220k_Estrategia_tipologias_normalizada.xlsx']

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = "Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221130" #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

In [ ]:
# Base de seguimiento
temporal.to_csv(path+'Seguimiento_ETB_Total_dic.csv')

### Estrategia de duplicados en diciembre

In [ ]:
temporal = pd.read_csv(path+'Seguimiento_ETB_Total_dic.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,46,52,53,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
temporal.shape

(1000773, 73)

In [ ]:
df_m = temporal[temporal.TELEFONO.isna()]
# df_m.reset_index(drop=True, inplace=True)

In [ ]:
df_m.shape

(224134, 73)

In [ ]:
df_t_= temporal[temporal.TELEFONO.notna()]
df_t_.reset_index(drop=True, inplace=True)

In [ ]:
df_t_.shape

(776639, 73)

In [ ]:
dft_ct = df_t_[df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1][df_t_.Emergia_Diciembre_Bajas!=1][df_t_.Emergia_Diciembre_nuevo!=1]
dft_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = df_m['NUMERO_DOCUMENTO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)), assume_unique=True))[0])
print('Cantidad de registros iguales entre la base actual: ', len(ind_doc))
indd = df_m.drop(df_m.index[ind_doc], axis=0).index
indd = indd[:39000]

<ipython-input-199-04880230d4c0>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct = df_t_[df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1][df_t_.Emergia_Diciembre_Bajas!=1][df_t_.Emergia_Diciembre_nuevo!=1]


Cantidad de registros iguales entre la base actual:  180522


In [ ]:
temporal['gestion_diciembre'].iloc[indd] = 1

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
temporal[temporal.TELEFONO.isna()]['gestion_diciembre'].sum()

39000

In [ ]:
temporal.gestion_diciembre.sum()

211143

In [ ]:
df_m.shape[0]-len(ind_doc)

42890

In [ ]:
# Datos organizados de 39k para el seguimiento de contactos con el teléfono en NaN
data_ = df_m.drop(ind_doc)
data_.shape

(42890, 73)

In [ ]:
data_.reset_index(drop=True, inplace=True)
data_f = data_.iloc[:39000,:34].copy()
data_f = check(data_f)
data_f = main(data_f)

cantidad de registros: 39000 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 0 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 13830 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 38255 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 23426 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 13830 

cantidad de registros telefono 2: 38255 

cantidad de registros telefono 3: 23426 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 13830 

cantidad d

In [ ]:
data_f.to_excel(path+'Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221230_39k_registros_organizados.xlsx')

In [ ]:
dft_st = df_t_[df_t_.TELEFONO_CONTACTO_1.isna()]
dft_ct = df_t_[df_t_.TELEFONO_CONTACTO_1.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1][df_t_.Emergia_Diciembre_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)

dfm_st = df_m[df_m.TELEFONO_CONTACTO_1.isna()]
dfm_ct = df_m[df_m.TELEFONO_CONTACTO_1.notna()]
dfm_ct.reset_index(drop = True, inplace= True)

x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO_CONTACTO_1'].tolist()
y2 = dfm_ct['TELEFONO_CONTACTO_1'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print('Cantidad de registros iguales entre la base actual: ', len(indx))

<ipython-input-108-2916f660424f>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct = df_t_[df_t_.TELEFONO_CONTACTO_1.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1][df_t_.Emergia_Diciembre_Bajas!=1]


Cantidad de registros iguales entre la base actual:  80829


In [ ]:
df_m.shape[0]-112153-80829

31152

In [ ]:
dfm_ct.drop(indx).shape

(31152, 73)

In [ ]:
39000-31152-7848

0

In [ ]:
dfm_st.iloc[:7848,:].shape

(7848, 73)

In [ ]:
dft_st = df_t_[df_t_.TELEFONO_CONTACTO_1.isna()]
dft_ct = df_t_[df_t_.TELEFONO_CONTACTO_1.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]
# dft_ct.reset_index(drop = True, inplace= True)

dft_ct_ = df_t_[df_t_.TELEFONO_CONTACTO_2.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]
# dft_ct_.reset_index(drop = True, inplace= True)

dft_ct_1 = df_t_[df_t_.TELEFONO_CONTACTO_3.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]
# dft_ct_1.reset_index(drop = True, inplace= True)

dfm_st = df_m[df_m.TELEFONO_CONTACTO_1.isna()]
dfm_ct = df_m[df_m.TELEFONO_CONTACTO_1.notna()]
# dfm_ct.reset_index(drop = True, inplace= True)

dfm_st = df_m[df_m.TELEFONO_CONTACTO_2.isna()]
dfm_ct_= df_m[df_m.TELEFONO_CONTACTO_2.notna()]
# dfm_ct_.reset_index(drop = True, inplace= True)

dfm_st = df_m[df_m.TELEFONO_CONTACTO_3.isna()]
dfm_ct_1= df_m[df_m.TELEFONO_CONTACTO_3.notna()]
# dfm_ct_1.reset_index(drop = True, inplace= True)

x1  = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1  = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2  = dft_ct['TELEFONO_CONTACTO_1'].tolist()
x22 = dft_ct_['TELEFONO_CONTACTO_2'].tolist()
x222= dft_ct_1['TELEFONO_CONTACTO_3'].tolist()
y2  = dfm_ct['TELEFONO_CONTACTO_1'].tolist()
y22 = dfm_ct_['TELEFONO_CONTACTO_2'].tolist()
y222= dfm_ct_1['TELEFONO_CONTACTO_3'].tolist()

ind_doc  = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])

ind_con  = list(np.where(np.isin(y2, list(set(x2) & set(y2)), assume_unique=True))[0])
ind_con1 = list(np.where(np.isin(y2, list(set(x2) & set(y22)), assume_unique=True))[0])
ind_con2 = list(np.where(np.isin(y2, list(set(x2) & set(y22)), assume_unique=True))[0])

ind_con3 = list(np.where(np.isin(y2, list(set(x22) & set(y2)), assume_unique=True))[0])
ind_con4 = list(np.where(np.isin(y2, list(set(x22) & set(y22)), assume_unique=True))[0])
ind_con5 = list(np.where(np.isin(y2, list(set(x22) & set(y22)), assume_unique=True))[0])

ind_con6 = list(np.where(np.isin(y2, list(set(x22) & set(y2)), assume_unique=True))[0])
ind_con7 = list(np.where(np.isin(y2, list(set(x22) & set(y22)), assume_unique=True))[0])
ind_con8 = list(np.where(np.isin(y2, list(set(x22) & set(y22)), assume_unique=True))[0])

print(len(ind_doc), len(ind_con), len(ind_con1), len(ind_con2), len(ind_con3), 
      len(ind_con4), len(ind_con5), len(ind_con6), len(ind_con7), len(ind_con8))
indx = list(set(ind_doc) & (set(ind_con) | set(ind_con1) | set(ind_con2)) & (set(ind_con3) | set(ind_con4) | set(ind_con5)) & (set(ind_con6) | set(ind_con7) | set(ind_con8)))
print('Cantidad de registros iguales entre la base actual y la base con NaN en Número de conexión: ', len(indx))
# verificamos la candi
# df_New = dfm_ct.drop(indx, axis=0)
# print('Registros nuevos: {}'.format(df_New.shape))

<ipython-input-28-4a014e515d84>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct = df_t_[df_t_.TELEFONO_CONTACTO_1.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]
<ipython-input-28-4a014e515d84>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct_ = df_t_[df_t_.TELEFONO_CONTACTO_2.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]
<ipython-input-28-4a014e515d84>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct_1 = df_t_[df_t_.TELEFONO_CONTACTO_3.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.E

68910 79956 75510 75510 75643 75613 75613 75643 75613 75613
Cantidad de registros iguales entre la base actual y la base con NaN en Número de conexión:  64360


In [ ]:
dft_ct = df_t_[df_t_.TELEFONO_CONTACTO_1.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]

<ipython-input-30-f80f4d7be1b9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct = df_t_[df_t_.TELEFONO_CONTACTO_1.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]


In [ ]:
list(set(x2) & set(y2))

# Enero

In [ ]:
Mes = 'Enero' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2023' #@param ['2022', '2023']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ### Verificación de archivos disponibles. { vertical-output: true }
os.listdir(path)

['Base Total de Clientes de la OperaciÃ³n Fija_20221228.xlsx',
 'Seguimiento_ETB_Total_dic.csv',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221228_55k_Estrategia_5cc.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221228_55k_Estrategia_5cc_normalizada.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221228_17k_nuevos.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221228_17k_nuevos_normalizada.xlsx',
 'mayorel_Enero_60k.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221228_148k_normalizada.xlsx',
 'Base Total de Clientes de la OperaciÃ³n Fija_20221230_39k_registros_organizados.xlsx',
 '01_23_Base_MIX_BJ_19K_JJ.xlsx',
 '01_23_BASE_MOVIL_BARBECHOS_Q2_NO_GEST.xlsx',
 '01_23_BASE_MOVIL_BARBECHOS_Q1_SI_GESTION.xlsx']

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = "Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221228" #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo)
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635019 entries, 0 to 635018
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   TIPO_DOCUMENTO         635019 non-null  object        
 1   NUMERO_DOCUMENTO       635019 non-null  object        
 2   NOMBRE_CLIENTE         634252 non-null  object        
 3   TELEFONO               591555 non-null  object        
 4   TELEFONO_CONTACTO_1    499312 non-null  float64       
 5   TELEFONO_CONTACTO_2    556480 non-null  float64       
 6   TELEFONO_CONTACTO_3    415759 non-null  float64       
 7   LOCALIDAD              578070 non-null  object        
 8   BARRIO                 559708 non-null  object        
 9   DIRECCION_INSTALACION  634926 non-null  object        
 10  LATITUD                575768 non-null  float64       
 11  LONGITUD               575768 non-null  float64       
 12  PAQUETE                635019 non-null  obje

In [ ]:
# Base de seguimiento
temporal = pd.read_csv(path+'Seguimiento_ETB_Total_dic.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,46,52,53,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_m['Antiguedad_meses'] = 0

In [ ]:
# Guardar 5 cc
estrategia_5cc = df_m[df_m.MOVIL_PLAN_ETB_ACTUAL.notna()]
# estrategia_5cc = estrategia_5cc[df_t_.columns[:34].tolist()]
estrategia_5cc.reset_index(drop=True, inplace=True)
estrategia_5cc.to_excel(path+'Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221228_55k_Estrategia_5cc.xlsx')
estrategia_5cc = check(estrategia_5cc)
estrategia_5cc = main(estrategia_5cc)
estrategia_5cc.to_excel(path+'Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221228_55k_Estrategia_5cc_normalizada.xlsx')

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/content/drive/.shortcut-targets-by-id/1JLLYaqX1Sv4GHvHgK0EamfdLUmYpXJ92/ETB/Codes/2022/Functions/Normalization/func_callout.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['NUMERO_CONEXION'] = X['NUMERO_CONEXION'].apply(lambda x: str(int(float(x))) if


cantidad de registros: 54447 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION


/content/drive/.shortcut-targets-by-id/1JLLYaqX1Sv4GHvHgK0EamfdLUmYpXJ92/ETB/Codes/2022/Functions/Normalization/func_callout.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['NUMERO_CONEXION'] = X.apply(NC, axis=1)
/content/drive/.shortcut-targets-by-id/1JLLYaqX1Sv4GHvHgK0EamfdLUmYpXJ92/ETB/Codes/2022/Functions/Normalization/func_callout.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['telefono_1'] = X['telefono_1'].apply(lambda x: str(int(float(x))) if (isinstance(x,(float,int)) and not(math

Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 50639 

Inicio del pre-proceso de la columna telefono_1


/content/drive/.shortcut-targets-by-id/1JLLYaqX1Sv4GHvHgK0EamfdLUmYpXJ92/ETB/Codes/2022/Functions/Normalization/func_callout.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['telefono_1'] = X['telefono_1'].apply(TF)
/content/drive/.shortcut-targets-by-id/1JLLYaqX1Sv4GHvHgK0EamfdLUmYpXJ92/ETB/Codes/2022/Functions/Normalization/func_callout.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['largo_telefono_1'] = X['telefono_1'].apply(lambda x: len(x) if not(pd.isnull(x)) else x)
/content/drive/.sh

...

/content/drive/.shortcut-targets-by-id/1JLLYaqX1Sv4GHvHgK0EamfdLUmYpXJ92/ETB/Codes/2022/Functions/Normalization/func_callout.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['telefono_3'] = X['telefono_3'].apply(TF)
/content/drive/.shortcut-targets-by-id/1JLLYaqX1Sv4GHvHgK0EamfdLUmYpXJ92/ETB/Codes/2022/Functions/Normalization/func_callout.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['largo_telefono_3'] = X['telefono_3'].apply(lambda x: len(x) if  not(pd.isnull(x)) else x)


Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 44335 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 45792 

cantidad de registros telefono 2: 52588 

cantidad de registros telefono 3: 44335 



/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 45792 

cantidad de registros telefono 2: 50622 

cantidad de registros telefono 3: 9714 

Inicio del pre-proceso de Duplicados de los celular
cantidad de registros celular 1: 19851 

cantidad de registros celular 2: 13800 

cantidad de registros celular 3: 8019 

Eliminación de Duplicidad en los celulares
Finalizacion del pre-proceso de Duplicidad en celulares
cantidad de registros celular 1: 19851 

cantidad de registros celular 2: 13800 

cantidad de registros celular 3: 8019 

Inicio del preproceso... etapa 3-> Completitud 

cantidad de registros incompletos: 7 

cantidad de registros: 54447 



/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
df_m['Antiguedad_meses'] = 0
df_t_ = temporal.copy()
df_m = df_m[df_m.MOVIL_PLAN_ETB_ACTUAL.isna()]
df_m = df_m[df_t_.columns[:34].tolist()]
df_m.shape

(580572, 34)

In [ ]:
df_m.drop_duplicates(subset=['TELEFONO','NUMERO_DOCUMENTO'], keep='first', inplace=True)
df_m.reset_index(drop=True, inplace=True)
df_m.shape

(580027, 34)

In [ ]:
# Guardar la información del mes correspondiente
# Nuevos
# data con mes Septiembre
dft_st = df_t_[(df_t_.TELEFONO.isna()) | (df_t_.TELEFONO=='nan')]
dft_ct = df_t_[(df_t_.TELEFONO.notna()) | (df_t_.TELEFONO!='nan')][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].astype(str).tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].astype(str).tolist()
x2 = dft_ct['TELEFONO'].astype(str).tolist()
y2 = dfm_ct['TELEFONO'].astype(str).tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc),len(ind_con),len(indx))
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

<ipython-input-36-ee94530ef767>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct = df_t_[(df_t_.TELEFONO.notna()) | (df_t_.TELEFONO!='nan')][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Octubre_Bajas!=1][df_t_.Emergia_Noviembre_Bajas!=1]


525571 523250 522972
Registros nuevos: (17574, 34)


In [ ]:
df_New.to_excel(path+'Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221228_17k_nuevos.xlsx')

In [ ]:
df_New_ = df_New.copy()
df_New_ = check(df_New_)
df_New_ = main(df_New_)
df_New_.to_excel(path+'Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221228_17k_nuevos_normalizada.xlsx')

cantidad de registros: 17574 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 17356 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 10358 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 16089 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 10075 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 10358 

cantidad de registros telefono 2: 16089 

cantidad de registros telefono 3: 10075 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 10358 

cantid

In [ ]:
dfm_st[[df_t_.columns[dfm_ct.columns.shape[0]:].values, 'Emergia_Diciembre_nuevo']] = 0
df_New[[df_t_.columns[df_New.columns.shape[0]:].values, 'Emergia_Diciembre_nuevo']] = 0
df_New['Emergia_Diciembre_nuevo'] = 1
df_t_['Emergia_Diciembre_nuevo'] = 0

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [ ]:
temporal = pd.DataFrame()
temporal = pd.concat([df_t_, df_New, dfm_st], axis=0)
temporal.reset_index(drop=True, inplace=True)
temporal.shape

(1000773, 70)

In [ ]:
dft_st = pd.concat([temporal[temporal.TELEFONO=='nan'], temporal[temporal.TELEFONO.isna()],temporal[temporal.Emergia_Mayo_Bajas==1],
                    temporal[temporal.Emergia_Junio_Bajas==1], temporal[temporal.Emergia_Julio_Bajas==1], 
                    temporal[temporal.Emergia_Agosto_Bajas==1], temporal[temporal.Emergia_Septiembre_Bajas==1],
                    temporal[temporal.Emergia_Octubre_Bajas==1], temporal[temporal.Emergia_Noviembre_Bajas==1]], axis=0)
dft_ct = temporal[(temporal.TELEFONO.notna()) & (temporal.TELEFONO!='nan')][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df_m[df_m.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].astype(str).tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].astype(str).tolist()
x2 = dft_ct['TELEFONO'].astype(str).tolist()
y2 = dfm_ct['TELEFONO'].astype(str).tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
# datos = dft_ct[['TELEFONO','NUMERO_DOCUMENTO']].iloc[indx]
# y1 = datos['NUMERO_DOCUMENTO'].tolist()
# y2 = datos['TELEFONO'].tolist()
# indx = df_t.query("NUMERO_DOCUMENTO == @y1 and TELEFONO == @y2").index
print(len(ind_doc),len(ind_con),len(indx))
# df_tot = dft_ct.iloc[indx]
df_Eli = dft_ct.drop(indx,axis=0)
print('Registros eliminados: {}'.format(df_Eli.shape))

<ipython-input-41-770fe5e3b2c7>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct = temporal[(temporal.TELEFONO.notna()) & (temporal.TELEFONO!='nan')][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1]


544009 541264 540938
Registros eliminados: (20188, 70)


In [ ]:
dft_ct['Emergia_Diciembre_Bajas'] = 0
dft_ct['Emergia_Diciembre_Bajas'].iloc[df_Eli.index] = 1
dft_st['Emergia_Diciembre_Bajas'] = 0

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
temporal = pd.concat([dft_ct, dft_st], axis=0)
temporal.shape

(1000773, 71)

In [ ]:
temporal.reset_index(drop=True, inplace=True)

In [ ]:
# Guardar los datos de gestion para el mes de enero
temporal['conteo_act_m'] = 0
temporal['conteo_act_m'] = temporal['conteo']+temporal.Emergia_Octubre_nuevo+temporal.gestion_noviembre+temporal.gestion_diciembre

In [ ]:
# Organizar el seguimiento
datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Noviembre_nuevo!=1][temporal.Emergia_Octubre_nuevo!=1][temporal.gestion_noviembre!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1][temporal.Emergia_Diciembre_Bajas!=1]
datos = datos[datos.gestion_noviembre!=1][datos.gestion_diciembre!=1][datos.Emergia_Diciembre_nuevo!=1]
datos.shape

<ipython-input-56-cc70f0debea6>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Noviembre_nuevo!=1][temporal.Emergia_Octubre_nuevo!=1][temporal.gestion_noviembre!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1][temporal.Emergia_Diciembre_Bajas!=1]
<ipython-input-56-cc70f0debea6>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datos = datos[datos.gestion_noviembre!=1][datos.gestion_diciembre!=1][datos.Emergia_Diciembre_nuevo!=1]


(218686, 72)

In [ ]:
datos.sort_values(by=['conteo_act_m'], ascending=False, inplace=True)

In [ ]:
datos.conteo_act_m.unique()

array([3., 2., 1.])

In [ ]:
datos.iloc[:60000,:].conteo_act_m.value_counts()

3.0    60000
Name: conteo_act_m, dtype: int64

In [ ]:
datos.iloc[:60000,:34].to_excel(path+'mayorel_Enero_60k.xlsx')

In [ ]:
datos.sort_values(by=['conteo_act_m'], ascending=True, inplace=True)

In [ ]:
datos.conteo_act_m.unique()

array([1., 2., 3.])

In [ ]:
data_ = datos.iloc[:148500,:34].copy()

In [ ]:
datos.iloc[:148500,:].conteo_act_m.value_counts()

2.0    83147
3.0    60699
1.0     4654
Name: conteo_act_m, dtype: int64

In [ ]:
data_ = check(data_)
data_ = main(data_)

cantidad de registros: 148500 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 148307 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 108322 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 128129 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 74204 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 108322 

cantidad de registros telefono 2: 128129 

cantidad de registros telefono 3: 74204 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 108322 


In [ ]:
data_.to_excel(path+'Base Total de Clientes de la OperaciA\u0303\xB3n Fija_20221228_148k_normalizada.xlsx')

In [ ]:
# Colocar gestion de los temporal cargados para el mes de enero
# Organizar el seguimiento
temporal['gestion_enero'] = 0
indx = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Noviembre_nuevo!=1][temporal.Emergia_Octubre_nuevo!=1][temporal.gestion_noviembre!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1][temporal.Emergia_Diciembre_Bajas!=1][temporal.gestion_noviembre!=1][temporal.gestion_diciembre!=1][temporal.Emergia_Diciembre_nuevo!=1].sort_values(by=['conteo_act_m'], ascending=False, inplace=True).index
# temporal['gestion_enero'].iloc[indx][:]

<ipython-input-87-4dbe9672b2ce>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indx = temporal[temporal.TELEFONO.notna()][temporal.TELEFONO!='nan'][temporal.Emergia_Noviembre_nuevo!=1][temporal.Emergia_Octubre_nuevo!=1][temporal.gestion_noviembre!=1][temporal.Emergia_Mayo_Bajas!=1][temporal.Emergia_Junio_Bajas!=1][temporal.Emergia_Julio_Bajas!=1][temporal.Emergia_Agosto_Bajas!=1][temporal.Emergia_Septiembre_Bajas!=1][temporal.Emergia_Octubre_Bajas!=1][temporal.Emergia_Noviembre_Bajas!=1][temporal.Emergia_Diciembre_Bajas!=1][temporal.gestion_noviembre!=1][temporal.gestion_diciembre!=1][temporal.Emergia_Diciembre_nuevo!=1].index
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs

## Estratégia para el mes de Febrero

In [ ]:
Mes = 'Febrero' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2023' #@param ['2022', '2023']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
# Guardar Actualización
temporal.to_csv(path+'Seguimiento_ETB_Total_ene.csv')

In [ ]:
#@title Verificar los toques realizados de los meses anteriores
temporal = pd.read_csv(path+'Seguimiento_ETB_Total_ene.csv')
temporal.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# temporal.drop(columns=['Unnamed: 0.1'], axis=1, inplace=True)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,46,52,53,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Verificar toques realizados de registros de agosto
temporal_agosto = pd.read_excel(path+'01_23_Base_MIX_BJ_19K_JJ.xlsx')
temporal_agosto.rename(columns={'NUMERO_DE_DOCUMENTO':'NUMERO_DOCUMENTO'}, inplace=True)

In [ ]:
df_t = temporal.copy()
df_m = temporal_agosto.copy()

In [ ]:
# Verificar toques de BBDD cargada anteiormente
dft_st = df_t[(df_t.TELEFONO.isna())]
dft_ct = df_t[(df_t.TELEFONO.notna())][df_t.Emergia_Mayo_Bajas!=1][df_t.Emergia_Junio_Bajas!=1][df_t.Emergia_Julio_Bajas!=1][df_t.Emergia_Agosto_Bajas!=1][df_t.Emergia_Septiembre_Bajas!=1][df_t.Emergia_Octubre_Bajas!=1][df_t.Emergia_Noviembre_Bajas!=1][df_t.Emergia_Diciembre_Bajas!=1]
dft_ct.reset_index(drop=True, inplace=True)
dft_ct = check(dft_ct)
dft_ct = main(dft_ct)

<ipython-input-7-a539f7c91b6b>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct = df_t[(df_t.TELEFONO.notna())][df_t.Emergia_Mayo_Bajas!=1][df_t.Emergia_Junio_Bajas!=1][df_t.Emergia_Julio_Bajas!=1][df_t.Emergia_Agosto_Bajas!=1][df_t.Emergia_Septiembre_Bajas!=1][df_t.Emergia_Octubre_Bajas!=1][df_t.Emergia_Noviembre_Bajas!=1][df_t.Emergia_Diciembre_Bajas!=1]


cantidad de registros: 540938 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 538739 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 380990 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 450250 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 268131 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 380990 

cantidad de registros telefono 2: 450250 

cantidad de registros telefono 3: 268131 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 380990

In [ ]:
df_m.NUMERO_CONEXION = df_m.NUMERO_CONEXION.astype(str)
df_m.NUMERO_CONEXION = df_m.NUMERO_CONEXION.apply(lambda x:x.replace('.0',''))
# df_m.NUMERO_CONEXION
df_m.NUMERO_CONEXION = df_m.NUMERO_CONEXION.apply(lambda x:x if len(x)==8 else np.nan)
dfm_st = df_m[df_m.NUMERO_CONEXION.isna()]
dfm_ct = df_m[df_m.NUMERO_CONEXION.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].astype(str).tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].astype(str).tolist()
x2 = dft_ct['NUMERO_CONEXION'].astype(str).tolist()
y2 = dfm_ct['NUMERO_CONEXION'].astype(str).tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc),len(ind_con),len(indx))

50676 17831 15262


In [ ]:
dft_st_ = df_t[(df_t.TELEFONO.isna())]
dft_ct_ = df_t[(df_t.TELEFONO.notna())][df_t.Emergia_Mayo_Bajas!=1][df_t.Emergia_Junio_Bajas!=1][df_t.Emergia_Julio_Bajas!=1][df_t.Emergia_Agosto_Bajas!=1][df_t.Emergia_Septiembre_Bajas!=1][df_t.Emergia_Octubre_Bajas!=1][df_t.Emergia_Noviembre_Bajas!=1][df_t.Emergia_Diciembre_Bajas!=1]
dft_ct_['gestion_agosto_dic'] = 0
dft_ct_['gestion_agosto_dic'][dft_ct_.index[dft_ct.index[indx]]] = 1

<ipython-input-9-715da1754ac0>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct_ = df_t[(df_t.TELEFONO.notna())][df_t.Emergia_Mayo_Bajas!=1][df_t.Emergia_Junio_Bajas!=1][df_t.Emergia_Julio_Bajas!=1][df_t.Emergia_Agosto_Bajas!=1][df_t.Emergia_Septiembre_Bajas!=1][df_t.Emergia_Octubre_Bajas!=1][df_t.Emergia_Noviembre_Bajas!=1][df_t.Emergia_Diciembre_Bajas!=1]
<ipython-input-9-715da1754ac0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft_ct_['gestion_agosto_dic'][dft_ct_.index[dft_ct.index[indx]]] = 1


In [ ]:
dft_st_['gestion_agosto_dic'] = 0

<ipython-input-10-fa8a674de9aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft_st_['gestion_agosto_dic'] = 0


In [ ]:
temporal_ = pd.concat([df_t[df_t.Emergia_Mayo_Bajas==1], df_t[df_t.Emergia_Junio_Bajas==1], 
                       df_t[df_t.Emergia_Julio_Bajas==1], df_t[df_t.Emergia_Agosto_Bajas==1],
                       df_t[df_t.Emergia_Septiembre_Bajas==1], df_t[df_t.Emergia_Octubre_Bajas==1],
                       df_t[df_t.Emergia_Noviembre_Bajas==1], df_t[df_t.Emergia_Diciembre_Bajas==1]], axis=0)

In [ ]:
temporal_['gestion_agosto_dic'] = 0

In [ ]:
temporal1 = pd.concat([dft_ct_, dft_st_, temporal_], axis=0)

In [ ]:
temporal1.shape

(1000773, 73)

In [ ]:
temporal_ges = pd.read_excel(path+'01_23_BASE_MOVIL_BARBECHOS_Q2_NO_GEST.xlsx')

In [ ]:
temporal_ges = pd.read_excel(path+'01_23_BASE_MOVIL_BARBECHOS_Q1_SI_GESTION.xlsx')

In [ ]:
df_t = temporal1.copy()
df_t.reset_index(drop=True, inplace=True)

In [ ]:
df_m = temporal_ges.copy()

In [ ]:
df_t_ = check(df_t)
df_t_ = main(df_t_)

cantidad de registros: 1000773 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 773008 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 629222 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 872263 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 517617 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 629222 

cantidad de registros telefono 2: 872263 

cantidad de registros telefono 3: 517617 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 62922

In [ ]:
df_m.TELEFONO_1 = df_m.TELEFONO_1.astype(str)
df_m.TELEFONO_1 = df_m.TELEFONO_1.apply(lambda x:x.replace('.0',''))
df_m.rename(columns={'NUMERO_DOCUMENTO_CLIENTE':'NUMERO_DOCUMENTO'}, inplace=True)
# df_m.TELEFONO_1

In [ ]:
dft_ct = df_t_[df_t_.gestion_diciembre==1]
dft_ct.reset_index(drop = True, inplace= True)
dfm_ct = df_m[df_m.TELEFONO_1.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['telefono_1'].astype(str).tolist()
y1 = dfm_ct['TELEFONO_1'].astype(str).tolist()
x2 = dft_ct['telefono_3'].astype(str).tolist()
y2 = dfm_ct['TELEFONO_3'].astype(str).tolist()

ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])

indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc), len(ind_con), len(indx))

83284 179856 73481


In [ ]:
# sin gestión
dft_ct = temporal1[temporal1.gestion_diciembre==1]
dft_st = temporal1[temporal1.gestion_diciembre!=1]
dft_ct['gestion_real_dic'] = 0
dft_ct['gestion_real_dic'].iloc[indx] = -1
dft_st['gestion_real_dic'] = 0

<ipython-input-45-9c1d272219ac>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft_ct['gestion_real_dic'] = 0
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-45-9c1d272219ac>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [ ]:
temporal2 = pd.concat([dft_ct, dft_st], axis=0)

In [ ]:
temporal2.shape

(1000773, 74)

In [ ]:
temporal2.conteo_act_m

In [ ]:
temporal2.to_csv(path+'Seguimiento_ETB_Total_ene_v2.csv')

In [ ]:
# Conteo de gestion
temporal2[['conteo_act_m', 'gestion_real_dic', 'gestion_agosto_dic']].sum(axis=1)

134811    3.0
134812    3.0
134813    2.0
134814    3.0
134815    3.0
         ... 
543547    0.0
543548    0.0
543549    0.0
543550    0.0
543551    0.0
Length: 1000773, dtype: float64

In [ ]:
temporal2[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][['conteo_act_m', 'gestion_real_dic', 'gestion_agosto_dic']].sum(axis=1).sort_values().value_counts()

<ipython-input-59-39aa522bb133>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][['conteo_act_m', 'gestion_real_dic', 'gestion_agosto_dic']].sum(axis=1).sort_values().value_counts()


3.0    238405
2.0    218488
4.0     33375
0.0     30644
1.0     17511
5.0      2500
6.0        15
dtype: int64

In [ ]:
temporal2[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1].shape

<ipython-input-50-1d86313dde35>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1].shape


(540938, 74)

In [ ]:
temporal2['conteo_act_m_enero'] = 0

In [ ]:
temporal2['conteo_act_m_enero'] = temporal2[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][['conteo_act_m', 'gestion_real_dic', 'gestion_agosto_dic']].sum(axis=1)

<ipython-input-80-c07d65496337>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2['conteo_act_m_enero'] = temporal2[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][['conteo_act_m', 'gestion_real_dic', 'gestion_agosto_dic']].sum(axis=1)


In [ ]:
temporal2.conteo_act_m_enero[(temporal2.conteo_act_m_enero[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Noviembre_nuevo==1]==0).index] = 1

<ipython-input-88-3f2c543d99d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2.conteo_act_m_enero[(temporal2.conteo_act_m_enero[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Noviembre_nuevo==1]==0).index] = 1


In [ ]:
temporal2.conteo_act_m_enero[(temporal2.conteo_act_m_enero[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Diciembre_nuevo==1]==0).index] = 1

<ipython-input-93-14f20ff7350d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2.conteo_act_m_enero[(temporal2.conteo_act_m_enero[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Diciembre_nuevo==1]==0).index] = 1


In [ ]:
temporal2.conteo_act_m_enero.value_counts()

3.0    238405
2.0    218488
1.0     48155
4.0     33375
5.0      2500
6.0        15
Name: conteo_act_m_enero, dtype: int64

In [ ]:
temporal2.conteo_act_m_enero[temporal2.conteo_act_m_enero[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.conteo_act_m_enero==0].index] = 2

<ipython-input-76-5df92170c208>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2.conteo_act_m_enero[temporal2.conteo_act_m_enero[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.conteo_act_m_enero==0].index] = 2


In [ ]:
temporal2.conteo_act_m_enero[(temporal2.TELEFONO.notna())][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1].sort_values().value_counts()

2.0    249132
3.0    238405
4.0     33375
1.0     17511
5.0      2500
6.0        15
Name: conteo_act_m_enero, dtype: int64

In [ ]:
temporal2.to_csv(path+'Seguimiento_ETB_Total_ene_v3.csv')

In [4]:
temporal2 = pd.read_csv(path+'Seguimiento_ETB_Total_ene_v3.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,46,52,53,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
temporal2.conteo_act_m_enero.value_counts()

3.0    238405
2.0    218488
1.0     48155
4.0     33375
5.0      2500
6.0        15
Name: conteo_act_m_enero, dtype: int64

In [54]:
temporal2.conteo.value_counts()

0.0    390908
2.0    299563
3.0    190733
1.0    119492
4.0        77
Name: conteo, dtype: int64

In [61]:
temporal2.conteo_act_m.value_counts()

3.0    130885
2.0     83147
1.0     17436
Name: conteo_act_m, dtype: int64

## Conteo de toques con barbechos de 60 días

In [162]:
temporal2.conteo_Octubre.sum()

138121

In [217]:
temporal2['gestion_enero'] = 0
temporal2['gestion_enero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.conteo_Octubre!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1].index] = 1

<ipython-input-217-e868ea9d20cd>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2['gestion_enero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.conteo_Octubre!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1].index] = 1
<ipython-input-217-e868ea9d20cd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2['gestion_enero'][temporal2[temporal2.TE

In [218]:
temporal2['Actual'] = 0
temporal2['Actual'] = temporal2.conteo_act_m_enero + temporal2['gestion_enero']

In [219]:
temporal2['Actual'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.conteo_Octubre!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_enero!=1].value_counts()

2.0    93054
3.0    19447
1.0    13207
4.0       18
Name: Actual, dtype: int64

In [220]:
temporal2['Actual'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.conteo_Octubre!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_enero!=1].value_counts().sum()

125726

In [221]:
temporal2['gestion_febrero'] = 0
temporal2['gestion_febrero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.conteo_Octubre!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_enero!=1].index] = 1

<ipython-input-221-30a44a9f4b07>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2['gestion_febrero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.conteo_Octubre!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_enero!=1].index] = 1
<ipython-input-221-30a44a9f4b07>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2['gestion_febrero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo

In [222]:
temporal2['gestion_febrero'].sum()

125726

In [223]:
temporal2['Actual'] = temporal2['Actual'] + temporal2['gestion_febrero']

In [233]:
temporal2['Actual'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_noviembre!=1][temporal2.gestion_diciembre!=1][temporal2.gestion_enero!=1][temporal2.gestion_febrero!=1].value_counts()

150903

In [227]:
temporal2['Actual'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.gestion_enero!=1][temporal2.gestion_febrero!=1].value_counts().sum()

150903

In [ ]:
temporal2['gestion_marzo'] = 0
temporal2['gestion_marzo'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.gestion_enero!=1][temporal2.gestion_febrero!=1].index] = 1

<ipython-input-75-a189f06e2191>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2['gestion_marzo'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.gestion_enero!=1][temporal2.gestion_febrero!=1].index] = 1
<ipython-input-75-a189f06e2191>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2['gestion_marzo'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=

In [ ]:
temporal2['Actual'] = temporal2['Actual'] + temporal2['gestion_marzo']

In [ ]:
temporal2['Actual'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.gestion_febrero!=1][temporal2.gestion_marzo!=1].value_counts()

4.0    119276
3.0     83180
2.0     17374
5.0     11638
Name: Actual, dtype: int64

## Conteo de toques con barbechos de 90 días

In [7]:
temporal2['gestion_enero'] = 0
temporal2['gestion_enero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1].index] = 1

<ipython-input-7-130fa544f103>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2['gestion_enero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1].index] = 1
<ipython-input-7-130fa544f103>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2['gestion_enero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia

In [ ]:
temporal2.columns

In [21]:
#cruce temporal para gestion Noviembre
Mes = 'Noviembre' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022', '2023']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [23]:
temporal_oct = pd.read_csv(path+'ETB_Total_cruce_octubre.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,35,41,44,45,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
temporal_nov = pd.read_csv(path+'Base_ETB_Total_Noviembre_138k_normalizada.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
temporal_nov.shape

(138121, 36)

In [ ]:
temporal2_ = check(temporal2.copy())
temporal2_ = main(temporal2_)

In [31]:
temporal_nov.columns

Index(['Unnamed: 0', 'TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NOMBRE_CLIENTE',
       'NUMERO_CONEXION', 'telefono_1', 'telefono_2', 'telefono_3',
       'LOCALIDAD', 'BARRIO', 'DIRECCION_INSTALACION', 'LATITUD', 'LONGITUD',
       'PAQUETE', 'PAQUETE_PLAN', 'CICLO', 'ESTADO', 'HABEAS_DATA',
       'MOVIL_ETB_ACTUAL', 'MOVIL_ETB_HISTORICO', 'Tipo_oferta',
       'Antiguedad_meses', 'VELOCIDAD', 'TECNOLOGIA', 'TARIFA_CONFIGURADA',
       'ESTRATO_USO', 'celular_1', 'celular_2', 'celular_3', 'C_SEG_UEN',
       'Localidad_zona', 'tipo_estrato', 'Tipo_velocidad', 'grupos', 'pqr',
       'conteo'],
      dtype='object')

In [80]:
temporal_nov.NUMERO_CONEXION = temporal_nov.NUMERO_CONEXION.astype(str)
temporal_nov.NUMERO_CONEXION = temporal_nov.NUMERO_CONEXION.apply(lambda x:x.replace('.0',''))
temporal_nov.NUMERO_CONEXION[temporal_nov.NUMERO_CONEXION!='nan'] = temporal_nov.NUMERO_CONEXION[temporal_nov.NUMERO_CONEXION!='nan'].astype(int)
temporal_nov.NUMERO_CONEXION[temporal_nov.NUMERO_CONEXION=='nan'] = np.nan

<ipython-input-80-bdc0d8188cd4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_nov.NUMERO_CONEXION[temporal_nov.NUMERO_CONEXION!='nan'] = temporal_nov.NUMERO_CONEXION[temporal_nov.NUMERO_CONEXION!='nan'].astype(int)
<ipython-input-80-bdc0d8188cd4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_nov.NUMERO_CONEXION[temporal_nov.NUMERO_CONEXION=='nan'] = np.nan


In [81]:
temporal2_.NUMERO_CONEXION = temporal2_.NUMERO_CONEXION.astype(str)
temporal2_.NUMERO_CONEXION = temporal2_.NUMERO_CONEXION.apply(lambda x:x.replace('.0',''))
temporal2_.NUMERO_CONEXION[temporal2_.NUMERO_CONEXION!='nan'] = temporal2_.NUMERO_CONEXION[temporal2_.NUMERO_CONEXION!='nan'].astype(int)
temporal2_.NUMERO_CONEXION[temporal2_.NUMERO_CONEXION=='nan'] = np.nan

<ipython-input-81-d96efe732485>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2_.NUMERO_CONEXION[temporal2_.NUMERO_CONEXION!='nan'] = temporal2_.NUMERO_CONEXION[temporal2_.NUMERO_CONEXION!='nan'].astype(int)
<ipython-input-81-d96efe732485>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2_.NUMERO_CONEXION[temporal2_.NUMERO_CONEXION=='nan'] = np.nan


In [87]:
dft_st = temporal2_[(temporal2_.NUMERO_CONEXION.isna())]
dft_ct = temporal2_[(temporal2_.NUMERO_CONEXION.notna())][temporal2_.Emergia_Mayo_Bajas!=1][temporal2_.Emergia_Junio_Bajas!=1][temporal2_.Emergia_Julio_Bajas!=1][temporal2_.Emergia_Agosto_Bajas!=1][temporal2_.Emergia_Septiembre_Bajas!=1][temporal2_.Emergia_Octubre_Bajas!=1][temporal2_.Emergia_Noviembre_Bajas!=1]
dft_ct.reset_index(drop=True, inplace=True)
dfm_ct = temporal_nov
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].astype(str).tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].astype(str).tolist()
x2 = dft_ct['NUMERO_CONEXION'].astype(str).tolist()
y2 = dfm_ct['NUMERO_CONEXION'].astype(str).tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc),len(ind_con),len(indx))

<ipython-input-87-e24c9b27e692>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft_ct = temporal2_[(temporal2_.NUMERO_CONEXION.notna())][temporal2_.Emergia_Mayo_Bajas!=1][temporal2_.Emergia_Junio_Bajas!=1][temporal2_.Emergia_Julio_Bajas!=1][temporal2_.Emergia_Agosto_Bajas!=1][temporal2_.Emergia_Septiembre_Bajas!=1][temporal2_.Emergia_Octubre_Bajas!=1][temporal2_.Emergia_Noviembre_Bajas!=1]


159287 124724 124016


In [90]:
temporal2['Gestion_oct'] = 0
temporal2['Gestion_oct'][(temporal2_.NUMERO_CONEXION.notna())][temporal2_.Emergia_Mayo_Bajas!=1][temporal2_.Emergia_Junio_Bajas!=1][temporal2_.Emergia_Julio_Bajas!=1][temporal2_.Emergia_Agosto_Bajas!=1][temporal2_.Emergia_Septiembre_Bajas!=1][temporal2_.Emergia_Octubre_Bajas!=1][temporal2_.Emergia_Noviembre_Bajas!=1].iloc[indx] = 1

In [72]:
temporal2[temporal2.conteo!=0][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Septiembre_nuevo!=1].sort_values(by=['conteo'])

<ipython-input-72-372b480a3314>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2[temporal2.conteo!=0][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Septiembre_nuevo!=1].sort_values(by=['conteo'])


,Unnamed: 0,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_CLIENTE,TELEFONO,TELEFONO_CONTACTO_1,TELEFONO_CONTACTO_2,TELEFONO_CONTACTO_3,LOCALIDAD,BARRIO,...,Emergia_Noviembre_nuevo,Emergia_Noviembre_Bajas,gestion_diciembre,Emergia_Diciembre_nuevo,Emergia_Diciembre_Bajas,conteo_act_m,gestion_agosto_dic,gestion_real_dic,conteo_act_m_enero,gestion_enero
319777,112763,CC,80878266,PEDRO ANTONIO AMAYA BENITES,13479712,NaN,3.057047e+09,NaN,USME,EL PROGRESO USME,...,0,0,0,0,0,1.0,0,0,1.0,1
259975,51124,CC,1073721715,MIGUEL ANGEL DIAZ SALAZAR,13587843,NaN,3.114438e+09,NaN,SOACHA,NaN,...,0,0,0,0,0,2.0,0,0,2.0,0
259976,51125,CC,19263497,ALVARO DE JESUS BARRETO ANGEL,12590712,6.012591e+09,3.158973e+09,3.158973e+09,USAQUEN,CEDRITOS,...,0,0,0,0,0,2.0,0,0,2.0,0
259977,51126,CC,1068579788,ARLETH JOHANA BALTAZAR ECHEVERRY,12318393,NaN,3.227325e+09,NaN,CIUDAD BOLIVAR,SUMAPAZ,...,0,0,0,0,0,2.0,0,0,2.0,0
259978,51127,NIT,-973747,HERREÑO ELISA TELLEZ DE,12901388,NaN,NaN,3.125529e+09,KENNEDY,HIPOTECHO SUR,...,0,0,0,0,0,2.0,0,0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166148,527454,CC,1014251642,ANNGIE STEPHANIA PEREZ RAMOS,17662338.0,6.017662e+09,3.184295e+09,NaN,CIUDAD BOLIVAR,ATLANTA,...,0,0,1,0,0,5.0,0,0,5.0,0
166149,527455,CC,51718964,LUZ MILA SOCHA AGUIRRE,16143364.0,6.016143e+09,3.103411e+09,NaN,USAQUEN,CEDRO SALAZAR,...,0,0,1,0,0,5.0,1,0,6.0,0
166150,527456,CC,1085246237,YEMARY QUINTERO SANCHEZ,12583907.0,6.012584e+09,3.045308e+09,NaN,USAQUEN,LA LIBERIA,...,0,0,1,0,0,5.0,0,0,5.0,0
166134,527440,CC,9738224,LUIS EDUARDO TAFUR HERRERA,17928713.0,6.017929e+09,3.108245e+09,3.108245e+09,CHAPINERO,MARIA CRISTINA,...,0,0,1,0,0,5.0,1,0,6.0,0


In [111]:
temporal2['conteo_Octubre'] = 0
temporal2['conteo_Octubre'][temporal2['conteo'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.Gestion_oct!=1][:138121].index]=1

<ipython-input-111-00af555fae91>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2['conteo_Octubre'][temporal2['conteo'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.Gestion_oct!=1][:138121].index]=1


In [115]:
temporal2['conteo_act_m_enero'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.Gestion_oct!=1][temporal2['conteo_Octubre']!=1].shape[0]

80565

In [149]:
temporal2['gestion_enero'] = 0
temporal2['gestion_enero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.Gestion_oct!=1][temporal2['conteo_Octubre']!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1].index] = 1

<ipython-input-149-d6c63e9d4fca>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2['gestion_enero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.Gestion_oct!=1][temporal2['conteo_Octubre']!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1].index] = 1
<ipython-input-149-d6c63e9d4fca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2['gestion_e

In [151]:
temporal2.conteo_act_m_enero[temporal2.conteo_act_m_enero.notna()].shape

(540938,)

In [135]:
temporal2['Actual'] = 0
temporal2['Actual'] = temporal2.conteo_act_m_enero + temporal2['gestion_enero']

In [136]:
temporal2['Actual'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Octubre_nuevo!=1][temporal2.Gestion_oct!=1][temporal2['conteo_Octubre']!=1][temporal2.gestion_noviembre!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.Emergia_Diciembre_nuevo!=1].value_counts()

4.0    74354
5.0     6211
Name: Actual, dtype: int64

In [137]:
temporal2['gestion_febrero'] = 0
temporal2['gestion_febrero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.gestion_enero!=1][temporal2.gestion_noviembre!=1].index] = 1

<ipython-input-137-dcb5a39ddc1f>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temporal2['gestion_febrero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.gestion_enero!=1][temporal2.gestion_noviembre!=1].index] = 1
<ipython-input-137-dcb5a39ddc1f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal2['gestion_febrero'][temporal2[temporal2.TELEFONO.notna()][temporal2.Emergia

In [146]:
temporal2['Actual'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Noviembre_nuevo!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.Emergia_Diciembre_nuevo!=1][temporal2.gestion_diciembre!=1][temporal2.gestion_enero!=1][temporal2.gestion_noviembre!=1].value_counts()

150903

In [138]:
temporal2['Actual'] = temporal2['Actual'] + temporal2['gestion_febrero']

In [132]:
temporal2['gestion_febrero'].sum()

263422

In [123]:
temporal2['Actual'] = temporal2['Actual'] + temporal2['gestion_febrero']

In [ ]:
temporal2['Actual'][temporal2.TELEFONO.notna()][temporal2.Emergia_Mayo_Bajas!=1][temporal2.Emergia_Junio_Bajas!=1][temporal2.Emergia_Julio_Bajas!=1][temporal2.Emergia_Agosto_Bajas!=1][temporal2.Emergia_Septiembre_Bajas!=1][temporal2.Emergia_Octubre_Bajas!=1][temporal2.Emergia_Noviembre_Bajas!=1][temporal2.Emergia_Diciembre_Bajas!=1][temporal2.gestion_enero!=1][temporal2.gestion_febrero!=1].value_counts()

In [17]:
temporal2.gestion_real_dic.sum() # conteo_act_m_enero

-73481

In [12]:
temporal2.gestion_diciembre.sum() + temporal2.gestion_real_dic.sum()

137662

In [ ]:
temporal2[temporal2.]

# Febrero

In [3]:
Mes = 'Febrero' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2023' #@param ['2022', '2023']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

# Gestión mayorel

In [ ]:
path_m = '/content/drive/MyDrive/ETB/Data/Otros proveedores - sabanas de retorno/2022/'
df_mayorel = pd.read_csv(path_m+'Gestion_mayorel_mayo_agosto.csv')
df_mayorel.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
df_mayorel.rename(columns={'NUMERO_DOCUMENTO_CLIENTE':'NUMERO_DOCUMENTO', 'NUMERO DE CONEXION':'TELEFONO'},inplace=True)

In [ ]:
df_mayorel['TELEFONO'] = df_mayorel['TELEFONO'].astype(str)
df_mayorel["TELEFONO"] = df_mayorel["TELEFONO"].apply(lambda x: x.replace(".0",""))

In [ ]:
df_mayorel.head()

,NUMERO_DOCUMENTO,TELEFONO,Mayorel_Gestionado_Mayo,Mayorel_NoContactado_Mayo,Mayorel_Venta_Mayo,Mayorel_Gestionado_Junio,Mayorel_NoContactado_Junio,Mayorel_Venta_Junio,Mayorel_Gestionado_Julio,Mayorel_NoContactado_Julio,Mayorel_Venta_Julio,Mayorel_Gestionado_Agosto,Mayorel_NoContactado_Agosto,Mayorel_Venta_Agosto
0,2863,13560217,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8126,16190087,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8997,16137856,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13340,16254071,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19865,12623167,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ges_m = ['Mayorel_Gestionado_Mayo', 'Mayorel_Gestionado_Junio', 'Mayorel_Gestionado_Julio', 'Mayorel_Gestionado_Agosto']
Data_m = pd.DataFrame()
Data_m['data'] = df_mayorel[ges_m].sum(axis=1)

In [ ]:
Data_m.groupby(by=['data'])['data'].count()

data
1.0    189650
Name: data, dtype: int64

In [ ]:
ges_e_m =['Emergia_Mayo_Gestion', 'Emergia_Junio_Gestion', 'Emergia_Julio_Gestion', 'Emergia_Agosto_Gestion', 'Emergia_Septiembre_nuevo',
          'Mayorel_Gestionado_Mayo', 'Mayorel_Gestionado_Junio', 'Mayorel_Gestionado_Julio', 'Mayorel_Gestionado_Agosto']
Data_m = pd.DataFrame()
temporal_1_ = temporal_1[temporal_1.TELEFONO.notna()]
Data_m['data'] = temporal_1_[ges_e_m].sum(axis=1)
Data_m.groupby(by=['data'])['data'].count()

In [ ]:
ges_e_m =['Emergia_Mayo_Gestion', 'Emergia_Junio_Gestion', 'Emergia_Julio_Gestion', 'Emergia_Agosto_Gestion', 'Emergia_Septiembre_nuevo',
          'Mayorel_Gestionado_Mayo', 'Mayorel_Gestionado_Junio', 'Mayorel_Gestionado_Julio', 'Mayorel_Gestionado_Agosto']
Data_m = pd.DataFrame()
temporal_1_ = temporal_1[temporal_1.TELEFONO.notna()]
Data_m['data'] = temporal_1_[ges_e_m].sum(axis=1)
Data_m.groupby(by=['data'])['data'].count()

In [ ]:
df_mayorel.Mayorel_Gestionado_Agosto.sum()

59325.0

In [ ]:
df_t_ = check(df_t_)

In [ ]:
df_t_ = main(df_t_)

cantidad de registros: 840817 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 722945 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 558211 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 718200 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 424326 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 558211 

cantidad de registros telefono 2: 718200 

cantidad de registros telefono 3: 424326 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 558211

In [ ]:
df_t_.columns

Index(['Unnamed: 0', 'TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NOMBRE_CLIENTE',
       'NUMERO_CONEXION', 'telefono_1', 'telefono_2', 'telefono_3',
       'LOCALIDAD', 'BARRIO', 'DIRECCION_INSTALACION', 'LATITUD', 'LONGITUD',
       'PAQUETE', 'PAQUETE_PLAN', 'CICLO', 'ESTADO', 'HABEAS_DATA',
       'MOVIL_ETB_ACTUAL', 'MOVIL_ETB_HISTORICO', 'Tipo_oferta',
       'Antiguedad_meses', 'VELOCIDAD', 'TECNOLOGIA', 'TARIFA_CONFIGURADA',
       'ESTRATO_USO', 'celular_1', 'celular_2', 'celular_3', 'C_SEG_UEN',
       'Localidad_zona', 'tipo_estrato', 'Tipo_velocidad', 'grupos', 'pqr',
       'Emergia_Mayo_nuevo', 'Emergia_Mayo_Bajas', 'Emergia_Mayo_Venta',
       'Emergia_Mayo_Gestion', 'Emergia_Mayo_NoContacto',
       'Emergia_Junio_nuevo', 'Emergia_Junio_Bajas', 'Emergia_Junio_Venta',
       'Emergia_Junio_Gestion', 'Emergia_Junio_NoContacto',
       'Emergia_Julio_nuevo', 'CORREO_ELECTRONICO', 'Emergia_Julio_Bajas',
       'Emergia_Julio_Venta', 'Emergia_Julio_Gestion',
       'Emergia_Julio

In [ ]:
df_t_.rename(columns={'NUMERO_CONEXION':'TELEFONO'}, inplace=True)

In [ ]:
df_t_.shape

(840817, 62)

In [ ]:
df_mayorel.drop(labels=['NUMERO_DOCUMENTO'], axis=1, inplace=True)

In [ ]:
del temporal_1

In [ ]:
df_mayorel.columns

Index(['TELEFONO', 'Mayorel_Gestionado_Mayo', 'Mayorel_NoContactado_Mayo',
       'Mayorel_Venta_Mayo', 'Mayorel_Gestionado_Junio',
       'Mayorel_NoContactado_Junio', 'Mayorel_Venta_Junio',
       'Mayorel_Gestionado_Julio', 'Mayorel_NoContactado_Julio',
       'Mayorel_Venta_Julio', 'Mayorel_Gestionado_Agosto',
       'Mayorel_NoContactado_Agosto', 'Mayorel_Venta_Agosto'],
      dtype='object')

In [ ]:
temporal_1 = df_t_.merge(df_mayorel, right_on=['TELEFONO'], left_on=['TELEFONO'], how='left')

In [ ]:
temporal_1.reset_index(drop=True, inplace=True)

In [ ]:
temporal_1.shape

(846510, 74)

In [ ]:
temporal_1[temporal_1.TELEFONO.notna()][temporal_1.Emergia_Mayo_Bajas!=1][temporal_1.Emergia_Junio_Bajas!=1][temporal_1.Emergia_Julio_Bajas!=1][temporal_1.Emergia_Agosto_Bajas!=1][temporal_1.Emergia_Septiembre_Bajas!=1].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(612290, 74)

In [ ]:
df_t_[df_t_.TELEFONO.isna()].shape

(115347, 62)

In [ ]:
temporal_1.columns

In [ ]:
temporal_1[temporal_1.TELEFONO.notna()].drop_duplicates().shape

(727960, 74)

In [ ]:
temporal_1[temporal_1.TELEFONO.notna()].drop_duplicates().shape

(727960, 74)

In [ ]:
tempo = pd.DataFrame()
tempo['TELEFONO'] = mayorel_sep.NUMERO_CONEXION.astype(str)
tempo['Mayorel_Gestionado_Septiembre'] = 1

In [ ]:
temporal_2 = temporal_1.merge(tempo, right_on=['TELEFONO'], left_on=['TELEFONO'], how='left')

In [ ]:
ges_e_m =['Emergia_Mayo_Gestion', 'Emergia_Junio_Gestion', 'Emergia_Julio_Gestion', 'Emergia_Agosto_Gestion', 'Emergia_Septiembre_nuevo',
          'Mayorel_Gestionado_Mayo', 'Mayorel_Gestionado_Junio', 'Mayorel_Gestionado_Julio', 'Mayorel_Gestionado_Agosto', 'Mayorel_Gestionado_Septiembre']
Data_m = pd.DataFrame()
temporal_1_ = temporal_2[temporal_2.TELEFONO.notna()][temporal_2.Emergia_Mayo_Bajas!=1][temporal_2.Emergia_Junio_Bajas!=1][temporal_2.Emergia_Julio_Bajas!=1][temporal_2.Emergia_Agosto_Bajas!=1][temporal_2.Emergia_Septiembre_Bajas!=1]
Data_m['data'] = temporal_1_[ges_e_m].sum(axis=1)
Data_m.groupby(by=['data'])['data'].count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


data
0.0     33898
1.0    322782
2.0    216947
3.0     51564
4.0         1
Name: data, dtype: int64

In [ ]:
ventas = ['Emergia_Mayo_Venta', 'Emergia_Junio_Venta', 'Emergia_Julio_Venta', 'Emergia_Agosto_Venta']
temporal_1_ = temporal_2[temporal_2.TELEFONO.notna()][temporal_2.TELEFONO.notna()][temporal_2.Emergia_Mayo_Bajas!=1][temporal_2.Emergia_Junio_Bajas!=1][temporal_2.Emergia_Julio_Bajas!=1][temporal_2.Emergia_Agosto_Bajas!=1][temporal_2.Emergia_Septiembre_Bajas!=1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
df_t_[ventas].sum().sum()

12542.0

In [ ]:
temporal_1 = temporal_1[pos]
temporal_1.shape

In [ ]:
ventas = ['Emergia_Mayo_Venta', 'Emergia_Junio_Venta', 'Emergia_Julio_Venta', 'Emergia_Agosto_Venta', 'Mayorel_Venta_Mayo', 'Mayorel_Venta_Junio', 'Mayorel_Venta_Julio', 'Mayorel_Venta_Agosto']
(temporal_1[ventas][pos].sum(axis=1)>0).sum()

12982

In [ ]:
nocontacto = ['Emergia_Mayo_NoContacto', 'Emergia_Junio_NoContacto', 'Emergia_Julio_NoContacto', 'Emergia_Agosto_NoContacto', 'Mayorel_NoContactado_Mayo', 'Mayorel_NoContactado_Junio','Mayorel_NoContactado_Julio', 'Mayorel_NoContactado_Agosto']
(temporal_1[nocontacto].sum(axis=1)>0).sum()

233410

In [ ]:
contacto = ['Emergia_Mayo_NoContacto', 'Emergia_Junio_NoContacto', 'Emergia_Julio_NoContacto', 'Emergia_Agosto_NoContacto', 'Mayorel_NoContactado_Mayo', 'Mayorel_NoContactado_Junio','Mayorel_NoContactado_Julio', 'Mayorel_NoContactado_Agosto']
(temporal_1[contacto].sum(axis=1)==0).sum()

607521

In [ ]:
Gest = ['Emergia_Mayo_Gestion', 'Emergia_Junio_Gestion', 'Emergia_Julio_Gestion', 'Emergia_Agosto_Gestion', 'Mayorel_Gestionado_Mayo', 'Mayorel_Gestionado_Junio', 'Mayorel_Gestionado_Julio', 'Mayorel_Gestionado_Agosto']
temporal_1[Gest][temporal_1[df_t_.columns].isna().sum(axis=1)!=60].sum().sum()

743918.0

In [ ]:
temporal_1 = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1][df_t_.Emergia_Septiembre_Bajas!=1][df_t_.Emergia_Septiembre_nuevo!=1].merge(df_mayorel, right_on=['TELEFONO', 'TELEFONO'], left_on=['TELEFONO', 'TELEFONO'], how='left')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
temporal_1.shape

(592963, 74)

In [ ]:
temporal_1[['Emergia_Mayo_Gestion','Emergia_Junio_Gestion','Emergia_Julio_Gestion','Emergia_Agosto_Gestion']].sum()

Emergia_Mayo_Gestion      140408.0
Emergia_Junio_Gestion     169070.0
Emergia_Julio_Gestion     110257.0
Emergia_Agosto_Gestion    190266.0
dtype: float64

In [ ]:
ventas = ['Emergia_Mayo_Venta', 'Emergia_Junio_Venta', 'Emergia_Julio_Venta', 'Emergia_Agosto_Venta', 'Mayorel_Venta_Mayo', 'Mayorel_Venta_Junio', 'Mayorel_Venta_Julio', 'Mayorel_Venta_Agosto']
(temporal_1[ventas].sum(axis=1)>0).sum()
# dft_ct = df_t_[df_t_.TELEFONO.notna()][df_t_.Emergia_Mayo_Bajas!=1][df_t_.Emergia_Junio_Bajas!=1][df_t_.Emergia_Julio_Bajas!=1][df_t_.Emergia_Agosto_Bajas!=1]


11448

In [ ]:
nocontacto = ['Emergia_Mayo_NoContacto', 'Emergia_Junio_NoContacto', 'Emergia_Julio_NoContacto', 'Emergia_Agosto_NoContacto', 'Mayorel_NoContactado_Mayo', 'Mayorel_NoContactado_Junio','Mayorel_NoContactado_Julio', 'Mayorel_NoContactado_Agosto']
(temporal_1[nocontacto].sum(axis=1)>0).sum()

307524

In [ ]:
Datos = temporal_1[temporal_1[nocontacto].sum(axis=1)>0]

In [ ]:
Datos.columns

Index(['Unnamed: 0', 'TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO', 'NOMBRE_CLIENTE',
       'TELEFONO', 'TELEFONO_CONTACTO_1', 'TELEFONO_CONTACTO_2',
       'TELEFONO_CONTACTO_3', 'LOCALIDAD', 'BARRIO', 'DIRECCION_INSTALACION',
       'LATITUD', 'LONGITUD', 'PAQUETE', 'PAQUETE_PLAN', 'CICLO', 'ESTADO',
       'HABEAS_DATA', 'MOVIL_ETB_ACTUAL', 'MOVIL_ETB_HISTORICO', 'Tipo_oferta',
       'Antiguedad_meses', 'VELOCIDAD', 'TECNOLOGIA', 'TARIFA_CONFIGURADA',
       'ESTRATO_USO', 'celular_1', 'celular_2', 'celular_3', 'C_SEG_UEN',
       'Localidad_zona', 'tipo_estrato', 'Tipo_velocidad', 'grupos', 'pqr',
       'Emergia_Mayo_nuevo', 'Emergia_Mayo_Bajas', 'Emergia_Mayo_Venta',
       'Emergia_Mayo_Gestion', 'Emergia_Mayo_NoContacto',
       'Emergia_Junio_nuevo', 'Emergia_Junio_Bajas', 'Emergia_Junio_Venta',
       'Emergia_Junio_Gestion', 'Emergia_Junio_NoContacto',
       'Emergia_Julio_nuevo', 'CORREO_ELECTRONICO', 'Emergia_Julio_Bajas',
       'Emergia_Julio_Venta', 'Emergia_Julio_Gestion',
 

In [ ]:
Eliminar = ['Unnamed: 0', 'Emergia_Mayo_nuevo', 'Emergia_Mayo_Bajas', 'Emergia_Mayo_Venta', 'Emergia_Mayo_Gestion', 'Emergia_Mayo_NoContacto',
            'Emergia_Junio_nuevo', 'Emergia_Junio_Bajas', 'Emergia_Junio_Venta', 'Emergia_Junio_Gestion', 'Emergia_Junio_NoContacto',
            'Emergia_Julio_nuevo', 'Emergia_Julio_Bajas', 'Emergia_Julio_Venta', 'Emergia_Julio_Gestion', 'Emergia_Julio_NoContacto',
            'Emergia_Agosto_nuevo', 'Emergia_Agosto_Bajas', 'Emergia_Agosto_Venta', 'Emergia_Agosto_Gestion', 'Emergia_Agosto_NoContacto',
            'Mayorel_Gestionado_Mayo', 'Mayorel_NoContactado_Mayo', 'Mayorel_Venta_Mayo',
            'Mayorel_Gestionado_Junio', 'Mayorel_NoContactado_Junio', 'Mayorel_Venta_Junio',
            'Mayorel_Gestionado_Julio', 'Mayorel_NoContactado_Julio', 'Mayorel_Venta_Julio',
            'Mayorel_Gestionado_Agosto', 'Mayorel_NoContactado_Agosto', 'Mayorel_Venta_Agosto']
Datos.drop(labels=Eliminar, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
Datos.shape

(248018, 41)

In [ ]:
Datos.to_excel(path+'ETB_Total_187k.xlsx')

In [ ]:
Datos = pd.read_excel(path+'ETB_Total_187k.xlsx')

In [ ]:
Datos.rename(columns={'TELEFONO':'telefono_1'}, inplace=True)

In [ ]:
Datos.rename(columns={'telefono_1':'TELEFONO'}, inplace=True)

In [ ]:
Datos.TELEFONO = Datos.TELEFONO.astype(str)

In [ ]:
Datos = check(Datos)

In [ ]:
Datos = main(Datos)

cantidad de registros: 187998 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 187971 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 132668 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 161719 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 87768 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 132668 

cantidad de registros telefono 2: 161719 

cantidad de registros telefono 3: 87768 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 132668 


In [ ]:
Datos.rename(columns={'Telefono':'TELEFONO'}, inplace=True)

In [ ]:
mayorel_sep = pd.read_excel(path+'Majorel_Septiembre.xlsx')
Emergia_sep = pd.read_excel(path+'Etb_total_septiembre.xlsx')

In [ ]:
mayorel_sep.shape

(96500, 37)

In [ ]:
mayorel_sep.NUMERO_CONEXION

In [ ]:
mayorel_sep.NUMERO_DOCUMENTO

In [ ]:
Datos.shape

(187998, 41)

In [ ]:
mayorel_sep.shape

(96500, 37)

In [ ]:
Mes = 'Octubre' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
Datos.shape

(187998, 42)

In [ ]:
Datos.reset_index(drop = True, inplace=True)

In [ ]:
mayorel_sep.NUMERO_CONEXION = mayorel_sep.NUMERO_CONEXION.astype(str)

In [ ]:
Datos.TELEFONO

In [ ]:
Emergia_sep.NUMERO_CONEXION = Emergia_sep.NUMERO_CONEXION.astype(str)

In [ ]:
Emergia_sep.NUMERO_CONEXION[0]

'17301595'

In [ ]:
Emergia_sep["NUMERO_CONEXION"] = Emergia_sep["NUMERO_CONEXION"].apply(lambda x: x.replace(".0",""))

In [ ]:
Emergia_sep.NUMERO_CONEXION[0]

In [ ]:
mayorel_sep["NUMERO_CONEXION"] = mayorel_sep["NUMERO_CONEXION"].apply(lambda x: x.replace(".0",""))

In [ ]:
mayorel_sep.NUMERO_CONEXION[0]

'17144024'

In [ ]:
# data con mes Septiembre
dft_st = Datos[Datos.NUMERO_CONEXION.isna()]
dft_ct = Datos[Datos.NUMERO_CONEXION.notna()]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = Emergia_sep[Emergia_sep.NUMERO_CONEXION.isna()]
dfm_ct = Emergia_sep[Emergia_sep.NUMERO_CONEXION.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['NUMERO_CONEXION'].tolist()
y2 = dfm_ct['NUMERO_CONEXION'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc),len(ind_con),len(indx))
df_New = dft_ct.drop(ind_doc,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

17479 50678 12906
Registros nuevos: (170492, 42)


In [ ]:
# data con mes Septiembre
dft_st = df_New[df_New.NUMERO_CONEXION.isna()]
dft_ct = df_New[df_New.NUMERO_CONEXION.notna()]
dft_ct.reset_index(drop=True, inplace=True)
dfm_st = mayorel_sep[mayorel_sep.NUMERO_CONEXION.isna()]
dfm_ct = mayorel_sep[mayorel_sep.NUMERO_CONEXION.notna()]
dfm_ct.reset_index(drop=True, inplace=True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['NUMERO_CONEXION'].tolist()
y2 = dfm_ct['NUMERO_CONEXION'].tolist()
ind_doc = list(np.where(np.isin(x1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc),len(ind_con),len(indx))
df_New_ = dft_ct.drop(ind_doc,axis=0)
print('Registros nuevos: {}'.format(df_New_.shape))

8283 29730 7749
Registros nuevos: (162209, 42)


In [ ]:
temporal_1.TELEFONO

0         13561955
1         17659219
2         13083282
3         13583572
4         16222389
            ...   
841326    17153001
841327    16269813
841328    12444939
841329    13652355
841330    12330881
Name: TELEFONO, Length: 841331, dtype: object

In [ ]:
df_New_.shape

(162209, 42)

In [ ]:
df_New_.NUMERO_CONEXION.reset_index(drop=True, inplace=True)

In [ ]:
df_New_.rename(columns={'NUMERO_CONEXION':'TELEFONO'}, inplace=True) 

In [ ]:
df_New_2 = df_New_[['TELEFONO']]

In [ ]:
df_New_2['Uso_sep'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
temporal_2 = temporal_1.merge(df_New_2, right_on=['TELEFONO'], left_on=['TELEFONO'], how='left')

In [ ]:
Corte_sep = temporal_2[temporal_2.TELEFONO.notna()][temporal_2.Emergia_Mayo_Bajas!=1][temporal_2.Emergia_Junio_Bajas!=1][temporal_2.Emergia_Julio_Bajas!=1][temporal_2.Emergia_Agosto_Bajas!=1][temporal_2.Emergia_Septiembre_Bajas!=1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
Emergia_sep_ = Emergia_sep[['NUMERO_CONEXION']]
mayorel_sep_ = mayorel_sep[['NUMERO_CONEXION']]

In [ ]:
Emergia_sep_['Uso_sep_emer'] = 1
mayorel_sep_['Uso_sep_mayorel'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
Emergia_sep_.rename(columns={'NUMERO_CONEXION':'TELEFONO'}, inplace=True) 
mayorel_sep_.rename(columns={'NUMERO_CONEXION':'TELEFONO'}, inplace=True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
Corte_sep = Corte_sep.merge(Emergia_sep_, right_on=['TELEFONO'], left_on=['TELEFONO'], how='left')

In [ ]:
Corte_sep = Corte_sep.merge(mayorel_sep_, right_on=['TELEFONO'], left_on=['TELEFONO'], how='left')

In [ ]:
Corte_sep.shape

(610373, 77)

In [ ]:
valores = Corte_sep[['Emergia_Mayo_Gestion','Emergia_Junio_Gestion','Emergia_Julio_Gestion','Emergia_Agosto_Gestion','Uso_sep',
           'Emergia_Mayo_nuevo','Emergia_Junio_nuevo','Emergia_Julio_nuevo','Emergia_Agosto_nuevo','Uso_sep_emer',
           'Uso_sep_mayorel']].sum(axis=1)

In [ ]:
valores.value_counts()

1.0    237107
2.0    220241
0.0    136054
3.0     16894
4.0        77
dtype: int64

In [ ]:
df_New_.TELEFONO

In [ ]:
temporal_1.merge()

In [ ]:
df_New_.rename(columns={'TELEFONO':'telefono_1'}, inplace=True) 

In [ ]:
df_New_.columns

In [ ]:
df_New_l = check(df_New_)

In [ ]:
df_New_1 = main(df_New_l)

In [ ]:
df_New_.NUMERO_CONEXION

In [ ]:
df_New_.to_excel(path+'ETB_Total_No_concactados_octubre_166k_3.xlsx')

In [ ]:
df_New_1.NUMERO_CONEXION.notna().sum()

0

In [ ]:
contacto = ['Emergia_Mayo_NoContacto', 'Emergia_Junio_NoContacto', 'Emergia_Julio_NoContacto', 'Emergia_Agosto_NoContacto', 'Mayorel_NoContactado_Mayo', 'Mayorel_NoContactado_Junio','Mayorel_NoContactado_Julio', 'Mayorel_NoContactado_Agosto']
(temporal_1[contacto].sum(axis=1)==0).sum()

402435

In [ ]:
Gest = ['Emergia_Mayo_Gestion', 'Emergia_Junio_Gestion', 'Emergia_Julio_Gestion', 'Emergia_Agosto_Gestion', 'Mayorel_Gestionado_Mayo', 'Mayorel_Gestionado_Junio', 'Mayorel_Gestionado_Julio', 'Mayorel_Gestionado_Agosto']
temporal_1[Gest][temporal_1[df_t_.columns].isna().sum(axis=1)!=60].sum().sum()

615561.0

In [ ]:
# temporal_1.to_csv(path+'Seguimiento_global_ETB_corte_Septiembre.csv')

In [ ]:
# temporal_1.to_excel(path+'Seguimiento_global_ETB_corte_Septiembre.xlsx')

In [ ]:
# temporal_1 = pd.read_csv(path+'Seguimiento_global_ETB_corte_Septiembre.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,46,52,53,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
fechas_= ['Emergia_Mayo_NoContacto', 'Emergia_Junio_NoContacto', 'Emergia_Julio_NoContacto', 'Emergia_Agosto_NoContacto'] 

In [ ]:
temporal_1[fechas][(temporal_1[fechas] == 0).sum(axis=1)>0].shape

(846386, 4)

In [ ]:
temporal_1.shape

(846386, 74)

In [ ]:
print('Gestion: ',temporal_1.Emergia_Agosto_Gestion.sum())
print('Ventas: ',temporal_1.Emergia_Agosto_Venta.sum())
print('NoContacto: ',temporal_1.Emergia_Agosto_NoContacto.sum())

Gestion:  216041.0
Ventas:  3873.0
NoContacto:  61971


# Pendiente

In [ ]:
#@title ###Verificador de parámetros de busqueda interna
dfo=check(dt_total)
print('Verificación terminada')
dfo.info()

Verificación terminada
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613133 entries, 0 to 612815
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   TIPO_DOCUMENTO         613133 non-null  object        
 1   NUMERO_DOCUMENTO       613133 non-null  object        
 2   NOMBRE_CLIENTE         612311 non-null  object        
 3   NUMERO_CONEXION        613133 non-null  object        
 4   telefono_1             461555 non-null  float64       
 5   telefono_2             509689 non-null  float64       
 6   telefono_3             306459 non-null  float64       
 7   LOCALIDAD              545813 non-null  object        
 8   BARRIO                 529813 non-null  object        
 9   DIRECCION_INSTALACION  613009 non-null  object        
 10  LATITUD                547393 non-null  float64       
 11  LONGITUD               547393 non-null  float64       
 12  PAQUETE              

In [ ]:
#@title ### Normalización
X_pre = main(dfo)

cantidad de registros: 613133 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 611166 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 438644 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 509157 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 304369 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 438644 

cantidad de registros telefono 2: 509157 

cantidad de registros telefono 3: 304369 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 438644

In [ ]:
#@title Guardar sabana { vertical-output: true, form-width: "600px" }
Nombre_adicional = "_actualizada_613K_normalizada" #@param {type:"string"}
Formato = '.xlsx' #@param ['.xlsx', '.csv']
if Formato == '.xlsx':
  X_pre.to_excel(path+nombre_completo[:-5]+Nombre_adicional+'.xlsx')
else:
  X_pre.to_cvs(path+nombre_completo[:-5]+Nombre_adicional+'.csv')

print('Sabana almacenada en {}, con el nombre de {}{}'.format(path,nombre_completo[:-5]+Nombre_adicional,Formato))

Sabana almacenada en Data/ETB_Total/2022/Agosto/, con el nombre de Base Total de Clientes de la Operacion Fija_20220829_actualizada_613K_normalizada.xlsx


# Verificador  

In [ ]:
path

'Data/ETB_Total/2022/Agosto/'

In [ ]:
Nombre_adicional = "_New_19K_normalizada"
print(path+nombre_completo[:-5]+Nombre_adicional)

Data/ETB_Total/2022/Agosto/Base Total de Clientes de la Operacion Fija_20220829_New_19K_normalizada


# validación mes septiembre

In [ ]:
Nombre_adicional = 'Base Total de Clientes de la Operacion Fija_20220829_actualizada_613K_normalizada'

In [ ]:
X_pre = pd.read_excel(path+Nombre_adicional+'.xlsx')

NameError: ignored

In [ ]:
X_pre.info()

In [ ]:
#@title ###Seleccionar mes para analizar {run: "auto", vertical-output: true }
Mes = 'Septiembre' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2022' #@param ['2022']
path = os.path.join('Data/ETB_Total/'+Year+'/'+Mes,'')

In [ ]:
#@title ###Carga del mes selecionado { run: "auto", vertical-output: true }
Nombre_archivo = 'Base Total ' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df_m = pd.read_excel(path+nombre_completo,sheet_name='Consulta1')
df_m.info()

In [ ]:
#@title ###Verificador de parámetros de busqueda interna
dfo=check(df_m)
print('Verificación terminada')
dfo.info()

In [ ]:
#@title ### Normalización
X_pre_m = main(dfo)

In [ ]:
X_pre['NUMERO_CONEXION'] = X_pre['NUMERO_CONEXION'].astype(str)
X_pre["NUMERO_CONEXION"] = X_pre["NUMERO_CONEXION"].apply(lambda x: x.replace(".0",""))
X_pre.shape

In [ ]:
X_pre_m.NUMERO_CONEXION[0]

In [ ]:
dft_st = X_pre[X_pre.NUMERO_CONEXION.isna()]
dft_ct = X_pre[X_pre.NUMERO_CONEXION.notna()]
dft_ct.reset_index(drop = True, inplace= True)
dfm_st = X_pre_m[X_pre_m.NUMERO_CONEXION.isna()]
dfm_ct = X_pre_m[X_pre_m.NUMERO_CONEXION.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['NUMERO_CONEXION'].tolist()
y2 = dfm_ct['NUMERO_CONEXION'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc),len(ind_con),len(indx))
# nuevos
df_New = dfm_ct.drop(indx,axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

In [ ]:
name = 'Base Total de Clientes de la OperaciÃ³n Fija_20220928_Nuevos_19k_normalizada'
df_New.to_excel(path+name+'.xls')

# Actuallización de la base de datos para reporte mensual

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.create('A new spreadsheet')

# Open our new sheet and add some data.
worksheet = gc.open('A new spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)